# **Team 5** 💣
A Jupyter Notebook Wrapper that combines the works of FollowYourPose & MMPose.

Brought to you by:
1. Maximus Lee
2. Aloysius Woo
3. Lim Huai Fu
4. Tan Ai Xin
5. Jin Zhenglong

### **NOTE:** To start, run all cells

## 🔧 **Setup** 🔧

### **Install Environment**
Est Time: 8 mins
- Python Environment
- GitHub Repo
- Folder Scaffolding
- FYP/MMPose Scaffolding

In [1]:
#@title Install Python and other required packages

def setup_python():
  %cd /content
  !sudo apt-get install python3.8
  !sudo apt-get install python3.8-distutils

  !update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.8 2
  !apt-get update
  !apt install software-properties-common
  !sudo dpkg --remove --force-remove-reinstreq python3-pip python3-setuptools python3-wheel
  !apt-get install python3-pip
  !apt-get install imagemagick
  !cat /etc/ImageMagick-6/policy.xml | sed 's/none/read,write/g'> /etc/ImageMagick-6/policy.xml
  !apt update

In [2]:
#@title Clone Repository

def setup_github(isColab):
  if isColab:
    %cd /content
    # Empty default folder
    !rm -rf /content/.config /content/sample_data

    !git clone https://github.com/maximus-lee-678/ict3104_team_05.git .
  else:
    %cd ..\FollowYourPose
    !mkdir checkpoints

    %cd checkpoints
    !git lfs install
    !git clone https://huggingface.co/YueMafighting/FollowYourPose_v1 .
    %cd ..\..\demos

In [3]:
#@title Load default YAMLs

def setup_default_yamls(isColab):
  if isColab:
    %cp -r /content/other_files/fyp_default_yamls/* /content/FollowYourPose/configs/
  else:
    !xcopy /s "..\other_files\fyp_default_yamls\*" "..\FollowYourPose\configs\"

In [4]:
#@title FollowYourPose/MMPose Libaries
# https://github.com/open-mmlab/mmpose
# https://blog.csdn.net/qq_21532607/article/details/130226728
# https://colab.research.google.com/github/open-mmlab/mmpose/blob/master/demo/MMPose_Tutorial.ipynb

def setup_dependency_lib():
  %cd /content
  !python3.8 -m pip install -r other_files/requirements/requirements_colab.txt

  !python3.8 -m mim install mmengine
  !python3.8 -m mim install mmpose
  !python3.8 -m mim install "mmcv>=2.0.0"
  !python3.8 -m mim install "mmdet>=3.0.0"

### **Drive Mounting**
Mount files from google drive
- checkpoints

In [5]:
# @title Google Account Selection
def mount_account_select():
  %cd /content
  !sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
  !sudo apt-get update -qq 2>&1 > /dev/null
  !sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
  !google-drive-ocamlfuse

In [6]:
# @title Mount Drive Contents

def mount_drive():
  !sudo apt-get install -qq w3m # to act as web browser
  !xdg-settings set default-web-browser w3m.desktop # to set default browser
  %cd /content
  !mkdir gdrive
  %cd gdrive
  !mkdir MyDrive
  %cd ..
  %cd ..
  !google-drive-ocamlfuse /content/gdrive/MyDrive

### **Project Dir Setup**
1. Clear sample files
2. Create directories
3. Import python libraries
4. Symbolic link existing folders

In [7]:
# @title Imports { display-mode: "form" }
ALLOWED_VIDEO_EXT = ['.mp4', '.avi', '.mov', '.mkv']

from pathlib import Path
import os
import sys
import psutil

import csv
import time
import datetime
from subprocess import check_output

import re
import yaml
import json
from enum import Enum

from ipywidgets import Dropdown, Output, Layout, widgets, Button, VBox, HBox
from IPython.display import display, Markdown, HTML, Video, Image, clear_output

from moviepy.editor import VideoFileClip, clips_array, TextClip, CompositeVideoClip, ColorClip
from moviepy.config import change_settings

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2
import imageio

In [8]:
# @title Init Path { display-mode: "form" }

# Check if running on google colab or locally
if os.getenv("COLAB_RELEASE_TAG"):
  # Check if the path is not already in sys.path before appending
  path_to_append = '/usr/local/lib/python3.8/dist-packages'
  if path_to_append not in sys.path:
      sys.path.append(path_to_append)

  change_settings({"IMAGEMAGICK_BINARY": "/usr/bin/convert"})

  print('RUNNING IN COLAB.')
else:
  # If running from local, we are current running in "demos" directory, move one level up
  if 'ROOT_DIR_PATH' not in globals():
     %cd ..

  print('RUNNING LOCALLY.')

if 'ROOT_DIR_PATH' not in globals():
    # Root Directory
    ROOT_DIR_PATH = os.getcwd()
else:
   %cd {ROOT_DIR_PATH}

# Video
VIDEO_DIR_PATH = Path(f"{ROOT_DIR_PATH}/video")
VIDEO_SKELETON_DIR_PATH = Path(f"{ROOT_DIR_PATH}/video/Skeleton")

# I/O Files
TRAINING_CONTENT_DIR_PATH = Path(f"{ROOT_DIR_PATH}/training_content")
CUSTOM_MODEL_DIR_PATH = Path(f"{ROOT_DIR_PATH}/custom_model")
INFERENCE_OUTPUT_DIR_PATH = Path(f"{ROOT_DIR_PATH}/inference_output")
TEST_OUTPUT_DIR_PATH = Path(f"{ROOT_DIR_PATH}/test_output")

MMPOSE_DIR_PATH = Path(f"{ROOT_DIR_PATH}/MMPose")
FYP_DIR_PATH = Path(f"{ROOT_DIR_PATH}/FollowYourPose")

# FYP
CONFIG_DIR_PATH = Path(f"{FYP_DIR_PATH}/configs")

CHECKPOINT_DIR_PATH = Path(f"{FYP_DIR_PATH}/checkpoints")

# Dataset Paths
CHARADES_LOOKUP_PATH = Path(f"{ROOT_DIR_PATH}/other_files/charades_lookup")
SIMS_LOOKUP_PATH = Path(f"{ROOT_DIR_PATH}/other_files/sims4action_lookup")

def create_dir_path():
  !mkdir {TRAINING_CONTENT_DIR_PATH}
  !mkdir {CUSTOM_MODEL_DIR_PATH}
  !mkdir {INFERENCE_OUTPUT_DIR_PATH}
  !mkdir {TEST_OUTPUT_DIR_PATH}

RUNNING IN COLAB.


In [9]:
# @title Symbolic Linking { display-mode: "form" }

def symbolic_linking():
  !mkdir /content/FollowYourPose/checkpoints

  # Checkpoints folder
  !ln -s /content/gdrive/MyDrive/checkpoints/* /content/FollowYourPose/checkpoints

# copy edition
# %cp -r /content/gdrive/MyDrive/checkpoints/* /content/FollowYourPose/checkpoints/

In [10]:
# @title Charades Data { display-mode: "form" }

# Charades Data Class from csv
class CharadesData:
  def __init__(self, row):
    id, subject, scene, quality, relevance, verified, script, objects, descriptions, actions, length = row
    self.id = id
    self.subject = subject
    self.scene = scene
    self.quality = quality
    self.relevance = relevance
    self.verified = verified
    self.script = script
    self.objects = objects.split(";")
    self.descriptions = descriptions
    self.length = length
    self.actions = {}

    # Convert actions in proper data structure ("class_id time_start time_end" -> class_id: [time_start, time_end])
    if len(actions) != 0:
      action_substrings = actions.split(';')
      for substring in action_substrings:
        parts = substring.split()
        key = parts[0]
        values = [self.convert_to_ms(parts[1]), self.convert_to_ms(parts[2])]
        self.actions[key] = values

  # For printing
  def __str__(self):
        return f"ID: {self.id}, Subject: {self.subject}, Scene: {self.scene}, Quality: {self.quality}, Relevance: {self.relevance}, Verified: {self.verified}, Script: {self.script}, Objects: {self.objects}, Descriptions: {self.descriptions}, Actions: {self.actions}, Length: {self.length}"

  # Helper function to convert time into ms
  def convert_to_ms(self, seconds):
    ss,ms = seconds.split('.')
    total_ms = 1000*int(ss) + int(ms)
    return total_ms

  # Caption getter with template
  def getCaption(self, index):
    if (self.actions):
      return f"In a {self.scene} setting, within the context of '{self.script}', the action '{action_descriptions[list(self.actions.keys())[index]]}' is taking place."
    return f"In a {self.scene} setting, the action '{self.script}' is taking place."

action_descriptions = {}
charades_all = []

def load_charades():
  # Load classes lookup table
  with open(Path(f"{CHARADES_LOOKUP_PATH}/Charades_v1_classes.txt"), 'r') as file:
      for line in file:
          code, description = line.strip().split(' ', 1)
          action_descriptions[code] = description

  # Load charades data A
  with open(Path(f"{CHARADES_LOOKUP_PATH}/Charades_v1_train.csv"), mode='r') as file:
      csv_reader = csv.reader(file)
      next(csv_reader, None)

      for row in csv_reader:
          charadeData = CharadesData(row)
          charades_all.append(charadeData)

  # Load charades data B
  with open(Path(f"{CHARADES_LOOKUP_PATH}/Charades_v1_test.csv"), mode='r') as file:
      csv_reader = csv.reader(file)
      next(csv_reader, None)

      for row in csv_reader:
          charadeData = CharadesData(row)
          charades_all.append(charadeData)


In [11]:
# @title Sims Data { display-mode: "form" }


# Charades Data Class from csv
class SimsData:
  def __init__(self, filename):
    pattern = r'^([^_]*)_S(\d*)([^_]*\d*)_(fC\d*|m\d*)'
    match = re.match(pattern, filename)

    self.isSims = True if match else False
    if self.isSims:
      self.activity = sims_const["activity"].get(match.group(1))
      self.subject = sims_const["subject"].get(match.group(2))
      self.scene = sims_const["scene"].get(match.group(3))
      self.camera = match.group(4)

      if (not self.activity or not self.subject or not self.scene or not self.camera):
        self.isSims = False

  # For printing
  def __str__(self):
      return f"Sims: {self.isSims}"

  # Caption getter with template
  def getCaption(self):
    return f"In a {self.scene} setting, {self.subject} is {self.activity}"

sims_const = {}
def load_sims():
  sims_const["subject"] = {
    "1": "a male character with a medium skin tone, wearing a black cap and a multicolored vest",
    "2": "a male character with pale skin and red hair, sporting glasses and a beige blazer over an orange shirt",
    "3": "a male character with a tan complexion, gray hair styled with a headband, and dressed in a light gray cardigan",
    "4": "a male character with a dark complexion, a shaved head, and wearing a white blazer over a gray, horizontally striped shirt",
    "5": "a female character with a medium complexion, straight black hair to the shoulders, wearing a dress with a distinct pattern",
    "6": "a female character with a medium complexion, sporting grey hair, orange glasses, and a black and white dress",
    "7": "a female character with a light to medium skin tone, featuring a shaved head, and wearing a beige wrap-style garment",
    "8": "a female character with a medium skin tone and hair in a ponytail, dressed in a light-colored, sleeveless top with a gentle pattern"
  }

  sims_const["scene"] = {
      "K1": "Kitchen",
      "K2": "Kitchen",
      "D1": "Dining Room",
      "D2": "Dining Room",
      "L1": "Living Room",
      "L2": "Living Room",
  }

  sims_const["activity"] = {
      "Co": "cooking",
      "Dr": "drinking",
      "Ea": "eating",
      "GS": "getting up and sitting Down",
      "RB": "reading book",
      "UC": "using computer",
      "UP": "using phone",
      "UT": "using tablet",
      "WA": "walking",
      "TV": "watching TV",
  }


### 🪟 **Windows** 🪟

In [12]:
# One-time
# Clone FYP inference base model

def windows_setup():
  %cd ..\FollowYourPose
  !mkdir checkpoints

  %cd checkpoints
  !git lfs install
  !git clone https://huggingface.co/YueMafighting/FollowYourPose_v1 .
  %cd ..\..\demos

  # Load default YAMLs
  !xcopy /s "..\other_files\fyp_default_yamls\*" "..\FollowYourPose\configs\"


## 🔍 **Data Exploration** 🔍

In [13]:
# @title Video Exploration { display-mode: "form" }

# Helper Functions
def getFolderContent(folder_name):
  subfolders = []
  for content in os.listdir(VIDEO_DIR_PATH):
        content_path = os.path.join(VIDEO_DIR_PATH, content)

        # Check if it's a directory and not hidden
        if os.path.isdir(content_path) and not content.startswith("."):
            subfolders.append(content)

  return subfolders

class VideoTab:

  def __init__(self, state):
    # Init
    self.state = state
    self.video_dir_folders = os.listdir(VIDEO_DIR_PATH)
    self.video_dir_folder_content = getFolderContent(self.video_dir_folders)

    # Create layout
    self.layout_single_button = widgets.Layout(width='212px',margin='3px 0px 0px 90px')
    self.layout_double_button = widgets.Layout(width='104px')
    self.layout_hbox = widgets.Layout(margin='0px 0px 0px 88px')
    self.layout_output = widgets.Layout(margin='0px 0px 20px 0px', display='flex', align_items='flex-start', height="532px")

    # Create widgets
    self.video_output_placeholder = widgets.Output(layout=self.layout_output)
    self.video_subdir_dropdown = widgets.Dropdown(options=self.video_dir_folder_content, description='Folder:', value=None, disabled=False)
    self.video_dropdown = widgets.Dropdown(options=[], description='Video:', disabled=True)
    self.video_display_button = widgets.Button(description="Display", disabled=True, layout=self.layout_single_button)
    self.video_refresh_button = widgets.Button(description="Refresh", disabled=False, layout=self.layout_single_button)
    self.video_mmpose_button = widgets.Button(description="MMPose", disabled=True, layout=self.layout_single_button)
    self.video_fyp_button = widgets.Button(description="FYP", disabled=True, layout=self.layout_single_button)
    self.video_train_button = widgets.Button(description="Train Model", disabled=True, layout=self.layout_single_button)
    self.video_test_button = widgets.Button(description="Test Model", disabled=True, layout=self.layout_single_button)

    self.video_hbox_1 = widgets.HBox([self.video_display_button, self.video_refresh_button], layout=self.layout_hbox)
    self.video_hbox_2 = widgets.HBox([self.video_mmpose_button, self.video_fyp_button], layout=self.layout_hbox)

    self.video_output_placeholder_content = widgets.HTML("""
      <div style="width: 512px; height: 512px; border-radius: 5%; background-color: black; margin: 0 auto; display: flex; justify-content: center; align-items: center;">
          <div style="width: 500px; height: 500px; border-radius: 5%; border: 2px solid white;" />
      </div>
    """)

  # Create listeners
  ## Update video dropdown options based on the selected folder
  def video_subdir_select(self, change):
      selected_video_folder = self.video_subdir_dropdown.value

      if selected_video_folder != None:
        selected_VIDEO_DIR_PATH = Path(f"{VIDEO_DIR_PATH}/{selected_video_folder}")
        selected_video_dir_content = [file for file in os.listdir(selected_VIDEO_DIR_PATH) if any(file.endswith(ext) for ext in ALLOWED_VIDEO_EXT) and not file.startswith('.')]

      else:
        selected_video_dir_content = []

      self.video_dropdown.options = selected_video_dir_content
      if not selected_video_dir_content:
          self.video_dropdown.disabled = False
          self.video_dropdown.value = None
      else:
          self.video_dropdown.disabled = False

      self.enable_folder_button()

  ## Display the selected video
  def display_selected_video(self, change):
      selected_video = self.video_dropdown.value
      selected_video_folder = self.video_subdir_dropdown.value

      if selected_video:
          video_path = Path(f"{VIDEO_DIR_PATH}/{selected_video_folder}/{selected_video}")
          video_display = Video(video_path, width=512, height=512, embed=True)

          # Clear the output placeholder and display the video
          with self.video_output_placeholder:
              clear_output(wait=True)
              display(video_display)

  ## Refresh folder and directory
  def refresh_folder_and_directory(self, change):
      video_dir_folders = os.listdir(VIDEO_DIR_PATH)
      video_dir_folder_content = getFolderContent(video_dir_folders)

      self.video_dropdown.options = []
      self.video_dropdown.value = None

      self.video_subdir_dropdown.options = video_dir_folder_content
      self.video_subdir_dropdown.value = None

  ## Enable button when a valid video is picked
  def enable_button(self, change):
      if self.video_dropdown.value:
          self.video_display_button.disabled = False
          self.video_mmpose_button.disabled = False
          self.video_fyp_button.disabled = False
      else:
          self.video_display_button.disabled = True
          self.video_mmpose_button.disabled = True
          self.video_fyp_button.disabled = True

  def enable_folder_button(self):
    if self.video_subdir_dropdown.value:
      self.video_train_button.disabled = False
      self.video_test_button.disabled = False
    else:
      self.video_train_button.disabled = True
      self.video_test_button.disabled = True

  def change_mmpose_state(self, change):
    content = [
        self.video_subdir_dropdown.value,
        self.video_dropdown.value
    ]
    self.state.change_state(State.MMPOSE, content)

  def change_fyp_state(self, change):
    content = [
        self.video_subdir_dropdown.value,
        self.video_dropdown.value
    ]
    self.state.change_state(State.FYP, content)

  def change_train_state(self, change):
    content = self.video_subdir_dropdown.value
    self.state.change_state(State.TRAIN, content)

  def change_test_state(self, change):
    content = self.video_subdir_dropdown.value
    self.state.change_state(State.TEST, content)

  def attach_listeners(self):
    # Attach Listeners
    self.video_subdir_dropdown.observe(self.video_subdir_select, 'value')
    self.video_display_button.on_click(self.display_selected_video)
    self.video_refresh_button.on_click(self.refresh_folder_and_directory)
    self.video_dropdown.observe(self.enable_button, 'value')
    self.video_mmpose_button.on_click(self.change_mmpose_state)
    self.video_fyp_button.on_click(self.change_fyp_state)
    self.video_train_button.on_click(self.change_train_state)
    self.video_test_button.on_click(self.change_test_state)

  def display_content(self):
    # Display fields
    hr = widgets.HTML("<hr>", layout=widgets.Layout(margin="3px 0px 0px 90px"))
    with self.video_output_placeholder:
        display(self.video_output_placeholder_content)

    with self.state.video_output:
      pane = widgets.VBox([
        self.video_subdir_dropdown,
        self.video_dropdown,
        self.video_refresh_button,
        self.video_display_button,
        hr,
        self.video_mmpose_button,
        self.video_fyp_button,
        self.video_train_button,
        self.video_test_button
      ])
      happy_box = widgets.HBox([self.video_output_placeholder, pane], layout=widgets.Layout(margin="10px 0px 10px 0px"))
      display(happy_box)



In [14]:
# @title Gif Exploration { display-mode: "form" }

# Helper Functions
def getInferenceFolderContent(folder_name):
  subfolders = []
  for content in os.listdir(INFERENCE_OUTPUT_DIR_PATH):
        content_path = os.path.join(INFERENCE_OUTPUT_DIR_PATH, content)

        # Check if it's a directory and not hidden
        if os.path.isdir(content_path) and not content.startswith("."):
            subfolders.append(content)

  return subfolders

class GifTab:

  def __init__(self, state):

    pose_type_options = ["Pose", "Human", "Human + Pose"]
    boolean_options = [True, False]

    # Init
    self.state = state
    self.gif_dir_folders = os.listdir(INFERENCE_OUTPUT_DIR_PATH)
    self.gif_dir_folder_content = getInferenceFolderContent(self.gif_dir_folders)

    # Create layout
    self.layout_single_button = widgets.Layout(margin='3px 0px 0px 210px', width="143px")
    self.layout_style = {'description_width': '200px'}
    self.layout_dropdown = widgets.Layout(width="350px")
    self.layout_hbox = widgets.Layout(margin='0px 0px 0px 88px')
    self.layout_output = widgets.Layout(margin='0px 0px 20px 0px', display='flex', align_items='flex-start', height="532px")

    # Create widgets
    self.gif_output_placeholder = widgets.Output(layout=self.layout_output)
    self.gif_subdir_dropdown = widgets.Dropdown(options=self.gif_dir_folder_content, description='Folder:', value=None, disabled=False, style=self.layout_style, layout=self.layout_dropdown)
    self.gif_display_button = widgets.Button(description="Display", disabled=True, layout=self.layout_single_button)
    self.gif_refresh_button = widgets.Button(description="Refresh", disabled=False, layout=self.layout_single_button)

    self.gif_pose_dropdown = widgets.Dropdown(options=pose_type_options, description='Pose Type:', value=None, disabled=False, style=self.layout_style, layout=self.layout_dropdown)
    self.gif_superimpose_dropdown = widgets.Dropdown(options=boolean_options, description='Display superimpose:', value=None, disabled=False, style=self.layout_style, layout=self.layout_dropdown)
    self.gif_captions_dropdown = widgets.Dropdown(options=boolean_options, description='Display captions:', value=None, disabled=False, style=self.layout_style, layout=self.layout_dropdown)
    self.gif_combine_dropdown = widgets.Dropdown(options=boolean_options, description='Combine all:', value=None, disabled=False, style=self.layout_style, layout=self.layout_dropdown)
    self.gif_generate_button = widgets.Button(description="Generate", disabled=True, layout=self.layout_single_button)
    self.gif_status_label = widgets.Label(value="", layout=self.layout_single_button)

    self.gif_output_placeholder_content = widgets.HTML("""
      <div style="width: 512px; height: 276px; border-radius: 5%; background-color: black; margin: 0 auto; display: flex; justify-content: center; align-items: center;">
          <div style="width: 500px; height: 270px; border-radius: 5%; border: 2px solid white;" />
      </div>
    """)

  # Create listeners
  ## Update video dropdown options based on the selected folder
  def gif_subdir_select(self, change):
      if self.gif_subdir_dropdown.value != None:
        self.enable_folder_button()

  ## Display the selected video
  def display_selected_gif(self, change):
      selected_inf_folder = self.gif_subdir_dropdown.value

      if selected_inf_folder:
          gif_path = Path(f"{INFERENCE_OUTPUT_DIR_PATH}/{selected_inf_folder}/processed")

          # Clear the output placeholder
          with self.gif_output_placeholder:
              clear_output(wait=True)

              # Display the GIF images in a VBox
              gif_widgets = []
              for gif in os.listdir(gif_path):
                  if gif.startswith("."):
                    continue

                  image_widget = Image(filename=Path(f"{gif_path}/{gif}"), format="gif", height=276)
                  output_widget = widgets.Output(layout=widgets.Layout(margin="0 0 6px 0"))

                  with output_widget:
                      display(image_widget)

                  gif_widgets.append(output_widget)

              gif_vbox = widgets.VBox(gif_widgets)
              display(gif_vbox)

  def process_gifs(self, change):

    self.gif_status_label.value = "Generating ..."
    inf_path = Path(f"{INFERENCE_OUTPUT_DIR_PATH}/{self.gif_subdir_dropdown.value}")
    pose_type = self.gif_pose_dropdown.value
    display_superimposed = self.gif_superimpose_dropdown.value
    display_captions = self.gif_captions_dropdown.value
    is_combine = self.gif_combine_dropdown.value

    postprocess_gif(inf_path, pose_type, display_superimposed, display_captions, is_combine)
    self.gif_status_label.value = "Done"

  ## Refresh folder and directory
  def refresh_folder_and_directory(self, change):
      gif_dir_folders = os.listdir(INFERENCE_OUTPUT_DIR_PATH)

      self.gif_subdir_dropdown.options = gif_dir_folders
      self.gif_subdir_dropdown.value = None

  def enable_folder_button(self):
    if self.gif_subdir_dropdown.value:
      self.gif_display_button.disabled = False
    else:
      self.gif_display_button.disabled = True

  def enable_generate_button(self, change):
    if (self.gif_subdir_dropdown.value
        and self.gif_pose_dropdown.value
        and self.gif_superimpose_dropdown.value != None
        and self.gif_captions_dropdown.value != None
        and self.gif_combine_dropdown.value != None):
      self.gif_generate_button.disabled = False
    else:
      self.gif_generate_button.disabled = True

  def attach_listeners(self):
    # Attach Listeners
    self.gif_subdir_dropdown.observe(self.gif_subdir_select, 'value')
    self.gif_pose_dropdown.observe(self.enable_generate_button, names='value')
    self.gif_superimpose_dropdown.observe(self.enable_generate_button, names='value')
    self.gif_captions_dropdown.observe(self.enable_generate_button, names='value')
    self.gif_combine_dropdown.observe(self.enable_generate_button, names='value')
    self.gif_display_button.on_click(self.display_selected_gif)
    self.gif_refresh_button.on_click(self.refresh_folder_and_directory)
    self.gif_generate_button.on_click(self.process_gifs)

  def display_content(self):
    # Display fields
    hr = widgets.HTML("<hr>", layout=widgets.Layout(margin="3px 0px 0px 90px"))
    with self.gif_output_placeholder:
        display(self.gif_output_placeholder_content)

    with self.state.gif_output:
      pane = widgets.VBox([
        self.gif_subdir_dropdown,
        self.gif_refresh_button,
        self.gif_display_button,
        hr,
        self.gif_pose_dropdown,
        self.gif_superimpose_dropdown,
        self.gif_captions_dropdown,
        self.gif_combine_dropdown,
        self.gif_generate_button,
        self.gif_status_label
      ])
      happy_box = widgets.HBox([self.gif_output_placeholder, pane], layout=widgets.Layout(margin="10px 0px 10px 0px"))
      display(happy_box)


In [15]:
# @title Gif Manipulation { display-mode: "form" }

def postprocess_gif(inf_path, pose_type, display_superimposed, display_captions, is_combine):

  clips = []
  raw_gifs = []

  for gif in os.listdir(Path(f"{inf_path}/raw")):
    if gif.endswith(".gif"):
      raw_gifs.append((gif, gif[:-4]))

  reader = imageio.get_reader(Path(f"{inf_path}/raw/{raw_gifs[0][0]}"))
  width, height, _ = reader.get_data(0).shape

  # Get pose type
  pose_type_dict = {
    "Pose": ("pose.gif", "P"),
    "Human": ("pose-human.gif", "H"),
    "Human + Pose": ("pose-bg.gif", "HP"),
  }
  pose_path = Path(f"{inf_path}/{pose_type_dict[pose_type][0]}")
  pose_video = VideoFileClip(pose_path.as_posix()).resize((width,height))

  black_clip = ColorClip(size=(pose_video.w, pose_video.h + 40), color=(0, 0, 0), duration=pose_video.duration)

  if display_captions:
    clips.append(CompositeVideoClip([black_clip, pose_video]))
  else:
    clips.append(pose_video)

  # Get clips
  video_path_template = "{}/superimposed/{}.gif" if display_superimposed else "{}/raw/{}.gif"
  for _, prompt in raw_gifs:
    file_path = Path(video_path_template.format(inf_path, prompt))
    gif_video = VideoFileClip(file_path.as_posix())
    if display_captions:
      txt_clip = TextClip(prompt, font="Amiri-bold", fontsize=30, color='white')
      txt_clip = txt_clip.set_duration(gif_video.duration)
      txt_clip = txt_clip.set_position(("center", "bottom"))
      components = [black_clip, txt_clip, gif_video]
      clips.append(CompositeVideoClip(components))
    else:
      clips.append(gif_video)

  suffix = f"{pose_type_dict[pose_type][1]}_{'S' if display_superimposed else 'X'}_{'C' if display_captions else 'X'}_{'C' if is_combine else 'X'}"

  if is_combine:
    gif_output_path = Path(f"{inf_path}/processed/all_combined_{suffix}.gif")
    result = clips_array([clips])
    result.write_gif(gif_output_path, fps=10, verbose=False, logger=None)
  else:
    for i, (path, prompt) in enumerate(raw_gifs):
      gif_output_path = Path(f"{inf_path}/processed/{prompt}_{suffix}.gif")
      result = clips_array([[clips[0], clips[i+1]]])
      result.write_gif(gif_output_path, fps=10, verbose=False, logger=None)


def postprocess_gif_test(model_name):

  model_path = Path(f"{TEST_OUTPUT_DIR_PATH}/{model_name}")
  for pose_name in os.listdir(model_path):
    pose_path = Path(f"{model_path}/{pose_name}")
    if not os.path.isdir(pose_path):
      continue

    raw_path = Path(f"{pose_path}/raw")
    superimpose_path = Path(f"{pose_path}/superimposed")
    pose_gif_path = Path(f"{pose_path}/pose.gif")
    flag = True

    for gif in os.listdir(raw_path):
      if gif.startswith("."):
        continue

      if flag:
        flag = False
        raw_path_first = Path(f"{pose_path}/raw/{gif}")
        reader = imageio.get_reader(raw_path_first)
        width, height, _ = reader.get_data(0).shape
        pose_gif_video = VideoFileClip(pose_gif_path.as_posix()).resize((width,height))

      clips = []
      clips.append(pose_gif_video)

      superimpose_gif_path = Path(f"{superimpose_path}/{gif}")
      superimpose_gif_video = VideoFileClip(superimpose_gif_path.as_posix())
      clips.append(superimpose_gif_video)

      raw_gif_path = Path(f"{raw_path}/{gif}")
      raw_gif_video = VideoFileClip(raw_gif_path.as_posix())
      clips.append(raw_gif_video)

      black_clip = ColorClip(size=(pose_gif_video.w * 3, pose_gif_video.h + 40), color=(0, 0, 0), duration=pose_gif_video.duration)
      txt_clip = TextClip(gif[:-4], font="Amiri-bold", fontsize=30, color='white')
      txt_clip = txt_clip.set_duration(pose_gif_video.duration)
      txt_clip = txt_clip.set_position(("center", "bottom"))

      gif_output_path = Path(f"{pose_path}/processed/{gif}")
      result = CompositeVideoClip([black_clip, clips_array([clips]), txt_clip])
      result.write_gif(gif_output_path, fps=7.692, verbose=False, logger=None)



In [16]:
# @title Test Grading { display-mode: "form" }

# Helper Functions
def getModelFolderContent():
  subfolders = []
  for content in os.listdir(TEST_OUTPUT_DIR_PATH):
        content_path = os.path.join(TEST_OUTPUT_DIR_PATH, content)

        # Check if it's a directory and not hidden
        if os.path.isdir(content_path) and not content.startswith("."):
            subfolders.append(content)

  return subfolders

class GradeTab:

  def __init__(self, state):

    # Init
    self.state = state
    self.model_dir_folders_content = getModelFolderContent()

    # Create layout
    self.layout_single_button = widgets.Layout(width="143px")
    self.layout_style = {'description_width': '50px'}
    self.layout_style_2 = {'description_width': '120px'}
    self.layout_dropdown = widgets.Layout(width="200px")
    self.layout_dropdown_2 = widgets.Layout(width="200px")
    self.layout_hbox = widgets.Layout(margin='0px 0px 0px 88px')
    self.layout_output = widgets.Layout(margin='0px 0px 20px 0px', display='flex', align_items='flex-start', height="532px")

    # Create widgets
    self.model_output_placeholder = widgets.Output(layout=self.layout_output)
    self.model_subdir_dropdown = widgets.Dropdown(options=self.model_dir_folders_content, description='Folder:', value=None, disabled=False, style=self.layout_style, layout=self.layout_dropdown)
    self.model_display_button = widgets.Button(description="Display", disabled=True, layout=self.layout_single_button)
    self.model_refresh_button = widgets.Button(description="Refresh", disabled=False, layout=self.layout_single_button)
    self.model_save_button = widgets.Button(description="Save", disabled=True, layout=self.layout_single_button)
    self.model_status_label = widgets.Label(value="", layout=self.layout_single_button)

    self.model_output_placeholder_content = widgets.HTML("""
      <div style="width: 512px; height: 276px; border-radius: 5%; background-color: black; margin: 0 auto; display: flex; justify-content: center; align-items: center;">
          <div style="width: 500px; height: 270px; border-radius: 5%; border: 2px solid white;" />
      </div>
    """)

    self.model_modules = []
    self.model_modules_metadata = []

  # Create listeners

  ## Display the selected video
  def display_selected_model(self, change):
      selected_model_folder = self.model_subdir_dropdown.value

      if selected_model_folder:
          self.model_status_label.value = "Loading display ..."
          model_pose_path = Path(f"{TEST_OUTPUT_DIR_PATH}/{selected_model_folder}")
          model_score_path = Path(f"{model_pose_path}/score.csv")

          score_dict = {}

          with open(model_score_path, newline='') as csvfile:
            csv_reader = csv.reader(csvfile)
            for row in csv_reader:
              key = f"{row[0]}-{row[1]}"
              score_dict[key] = row[2:]

          gif_widgets = []
          self.model_modules = []
          self.model_modules_metadata = []
          for pose in os.listdir(model_pose_path):
            pose_path = Path(f"{model_pose_path}/{pose}")
            if not os.path.isdir(pose_path) or pose.startswith("."):
              continue

            process_path = Path(f"{pose_path}/processed")
            for gif in os.listdir(process_path):
              if gif.startswith("."):
                continue

              img_bin = open(Path(f"{process_path}/{gif}"), 'rb').read()
              image_widget = widgets.Image(value=img_bin, format="gif", height=276, width=276*3)
              gif_widgets.append(image_widget)

              score_key = f"{pose}-{gif[:-4]}"
              score_data = score_dict.get(score_key)

              tb_pose = widgets.FloatText(description="Pose Score:", value="{:.2f}".format(float(score_data[0])) if score_data and score_data[0] else None, disabled=True, style=self.layout_style_2, layout=self.layout_dropdown_2)
              tb_env = widgets.IntSlider(
                  value=score_data[1] if score_data and score_data[1] else 4,
                  min=1,
                  max=7,
                  step=1,
                  disabled=False,
                  continuous_update=False,
                  orientation='horizontal',
                  readout=True,
                  readout_format='d',
                  description="Environment Score:",
                  style=self.layout_style_2,
                  layout=widgets.Layout(width="300px")
              )
              vbox = widgets.VBox([tb_pose, tb_env])

              self.model_modules_metadata.append((pose, gif[:-4]))
              self.model_modules.append(widgets.HBox([image_widget, vbox]))


          # Clear the output placeholder
          with self.model_output_placeholder:
              clear_output(wait=True)

              gif_vbox = widgets.VBox(self.model_modules)
              display(gif_vbox)

          self.model_status_label.value = ""

      if self.model_modules:
        self.model_save_button.disabled = False
      else:
        self.model_save_button.disabled = True

  def save_score(self, change):
    self.model_status_label.value = "Saving ..."
    selected_model_folder = self.model_subdir_dropdown.value
    model_pose_path = Path(f"{TEST_OUTPUT_DIR_PATH}/{selected_model_folder}")
    model_score_path = Path(f"{model_pose_path}/score.csv")

    headers = []
    score_dict = {}
    with open(model_score_path, newline='') as csvfile:
      csv_reader = csv.DictReader(csvfile)
      headers = csv_reader.fieldnames

      for row in csv_reader:
          key = f"{row['Pose']}-{row['Prompt']}"
          score_dict[key] = {
              "values": [row['Pose Score'], row['Environment Score']],
              "pose": row['Pose'],
              "prompt": row['Prompt']
          }

    for components, meta in zip(self.model_modules, self.model_modules_metadata):
      _, vbox = components.children
      _, env_score = vbox.children
      pose, prompt = meta

      score_key = f"{pose}-{prompt}"
      score_dict[score_key]["values"][1] = env_score.value


    score_list = [[entry["pose"], entry["prompt"]] + entry["values"] for entry in score_dict.values()]
    with open(model_score_path, 'w', newline='') as csvfile:
      csv_writer = csv.writer(csvfile)
      csv_writer.writerows([headers] + score_list)

    self.model_status_label.value = "Saved"
    print("Done")

  ## Refresh folder and directory
  def refresh_folder_and_directory(self, change):
      model_dir_folders = os.listdir(TEST_OUTPUT_DIR_PATH)

      self.model_subdir_dropdown.options = model_dir_folders
      self.model_subdir_dropdown.value = None

  def enable_folder_button(self, change):
    self.model_save_button.disabled = True
    if self.model_subdir_dropdown.value:
      self.model_display_button.disabled = False
    else:
      self.model_display_button.disabled = True

  def attach_listeners(self):
    # Attach Listeners
    self.model_subdir_dropdown.observe(self.enable_folder_button, 'value')
    self.model_display_button.on_click(self.display_selected_model)
    self.model_refresh_button.on_click(self.refresh_folder_and_directory)
    self.model_save_button.on_click(self.save_score)

  def display_content(self):
    # Display fields
    hr = widgets.HTML("<hr>", layout=widgets.Layout(margin="3px 0px 10px 0px"))
    with self.model_output_placeholder:
        display(self.model_output_placeholder_content)

    with self.state.grade_output:
      pane = widgets.HBox([
        self.model_subdir_dropdown,
        self.model_refresh_button,
        self.model_display_button,
        self.model_save_button,
        self.model_status_label
      ])
      vbox = widgets.VBox([pane, hr, self.model_output_placeholder], layout=widgets.Layout(margin="10px 0px 10px 0px"))
      display(vbox)


In [17]:
# @title Charts & Tables { display-mode: "form" }

class ChartTab:

  def __init__(self, state):

    # Init
    self.state = state

    # Create layout
    self.layout_single_button = widgets.Layout(width="143px")
    self.layout_style = {'description_width': '50px'}
    self.layout_style_2 = {'description_width': '120px'}
    self.layout_dropdown = widgets.Layout(width="200px")
    self.layout_dropdown_2 = widgets.Layout(width="200px")
    self.layout_hbox = widgets.Layout(margin='0px 0px 0px 88px')
    self.layout_output = widgets.Layout(margin='0px 0px 20px 0px', display='flex', align_items='flex-start', height="532px")

    # Create widgets
    self.chart_output_placeholder = widgets.Output(layout=self.layout_output)
    self.select_output_placeholder = widgets.Output(layout=self.layout_output)
    self.chart_refresh_button = widgets.Button(description="Refresh", disabled=False, layout=self.layout_single_button, button_style="info")
    self.chart_load_button = widgets.Button(description="Load", disabled=False, layout=self.layout_single_button, button_style="info")

    self.checkboxes = []
    self.all_scores = {}

  def load_csv(self):
    self.checkboxes = []
    self.all_scores = {}
    for model in os.listdir(TEST_OUTPUT_DIR_PATH):
      if model.startswith("."):
        continue

      model_score_csv = Path(f"{TEST_OUTPUT_DIR_PATH}/{model}/score.csv")
      if not model_score_csv.exists():
        continue

      df = pd.read_csv(model_score_csv)
      avg_pose_score = round(df['Score'].mean(), 2)
      avg_env_score = round(df['Manual Score'].mean(), 2)
      avg_score = round(avg_pose_score * 0.5 + avg_env_score * 10 * 0.5, 2)
      self.all_scores[model] = {
          "total_pose": len(df['Pose'].unique()),
          "avg_pose_score": avg_pose_score,
          "avg_env_score": avg_env_score,
          "avg_score": avg_score
      }

      checkbox = widgets.Checkbox(value=True, description=model, layout=Layout(margin='0px 0px 0px -80px'))
      checkbox.observe(self.update_chart, 'value')
      self.checkboxes.append(checkbox)

  def update_chart(self, *args):

      selected_names = [checkbox.description for checkbox in self.checkboxes if checkbox.value]
      if not selected_names:
        with self.select_output_placeholder:
          clear_output(wait=True)
          vbox = widgets.VBox([self.chart_refresh_button, widgets.Label(value="No model found")])
          display(vbox)
        return
      fig, axs = plt.subplots(3, 1, figsize=(6, 12))
      dict_pair = {
          "avg_score": "Average Score of Pose & Environment",
          "avg_pose_score": "Average Score of Pose",
          "avg_env_score": "Average Score of Environment"
      }

      # Function to add value labels on the bars
      def autolabel(rects):
          for rect in rects:
              height = rect.get_height()
              axs[i].annotate(f'{height}',
                              xy=(rect.get_x() + rect.get_width() / 2, height),
                              xytext=(0, 3),  # 3 points vertical offset
                              textcoords="offset points",
                              ha='center', va='bottom')

      for i, score_type in enumerate(dict_pair.keys()):
          selected_data = {name: self.all_scores[name][score_type] for name in selected_names}
          rects = axs[i].bar(selected_data.keys(), selected_data.values())
          autolabel(rects)  # Add value labels on top of bars

          axs[i].set_xlabel('Model')
          axs[i].set_ylabel('Score')
          axs[i].set_title(dict_pair[score_type])
          axs[i].set_ylim(0, max(selected_data.values()) * 1.2)

      plt.subplots_adjust(hspace=0.5)
      plt.tight_layout()

      output_chart = widgets.Output(layout=widgets.Layout(padding='20px 0 0 0'))
      with output_chart:
        plt.show()

      filtered_scores = {model: self.all_scores[model] for model in selected_names if model in self.all_scores}
      df = pd.DataFrame.from_dict(filtered_scores, orient='index')
      output_table = widgets.Output()
      with output_table:
        display(df)


      with self.chart_output_placeholder:
        clear_output(wait=True)
        vbox = widgets.VBox([output_table, output_chart])
        display(vbox)

      with self.select_output_placeholder:
        clear_output(wait=True)
        vbox = widgets.VBox([self.chart_refresh_button, widgets.VBox(self.checkboxes)])
        display(vbox)

  def load_charts(self, change):
    self.load_csv()
    self.update_chart()

  def attach_listeners(self):
    # Attach Listeners
    self.chart_load_button.on_click(self.load_charts)
    self.chart_refresh_button.on_click(self.load_charts)

  def display_content(self):
    # Display fields
    hr = widgets.HTML(
      "<div style='border-left: 2px solid; height: 100%; margin: 0 10px;'></div>"
    )

    with self.state.chart_output:
      pane = widgets.HBox([
      ])
      with self.select_output_placeholder:
        display(self.chart_load_button)

      hbox = widgets.HBox([self.chart_output_placeholder, hr, self.select_output_placeholder])
      vbox = widgets.VBox([pane, hbox], layout=widgets.Layout(margin="10px 0px 10px 0px"))
      display(vbox)


## 🦴 **MMPose** 🦴

In [18]:
# @title Inference { display-mode: "form" }

def run_mmpose_with_bg_inference(folder_path, video_name):
    full_path = Path(f"{VIDEO_DIR_PATH}/{folder_path}/{video_name}")

    %cd -q {MMPOSE_DIR_PATH}

    if not os.path.exists(VIDEO_SKELETON_DIR_PATH):
        os.mkdir(VIDEO_SKELETON_DIR_PATH)

    # Start inference
    !python demo/inferencer_demo.py \
        {full_path}  \
        --pose2d human \
        --vis-out-dir {VIDEO_SKELETON_DIR_PATH} \
        --thickness 4 \
        --radius 0

    # Change name for human background
    input_file = Path(f"{VIDEO_SKELETON_DIR_PATH}/{video_name}")
    bg_file = Path(f"{VIDEO_SKELETON_DIR_PATH}/.{video_name[:-4]}-bg.{video_name[-3:]}")

    os.rename(input_file, bg_file)

    return bg_file

def run_mmpose_inference(folder_path, video_name):
    full_path = Path(f"{VIDEO_DIR_PATH}/{folder_path}/{video_name}")

    %cd -q {MMPOSE_DIR_PATH}

    if not os.path.exists(VIDEO_SKELETON_DIR_PATH):
        os.mkdir(VIDEO_SKELETON_DIR_PATH)

    # Start inference
    !python demo/inferencer_demo.py \
        {full_path}  \
        --pose2d human \
        --vis-out-dir {VIDEO_SKELETON_DIR_PATH} \
        --black-background \
        --thickness 4 \
        --radius 0

    # Change name for human background
    input_file = Path(f"{VIDEO_SKELETON_DIR_PATH}/{video_name}")
    bg_file = Path(f"{VIDEO_SKELETON_DIR_PATH}/.{video_name[:-4]}-bg.{video_name[-3:]}")

    return input_file

In [44]:
# @title Video Encoding { display-mode: "form" }

# Helper Function to re-encode video due to H.264 video encoding error
def reencode_video(video_path):
  temp_output_file = Path(f"{VIDEO_SKELETON_DIR_PATH}/output.{(video_path.as_posix())[-3:]}")

  !ffmpeg -i {video_path} -c:v libx264 -crf 23 -c:a aac -strict experimental {temp_output_file}

  os.remove(video_path)
  os.rename(temp_output_file, video_path)

In [20]:
# @title Add metadata { display-mode: "form" }

def add_metadata_to_mp4(video_path, metadata):
    metadata_args = []
    for key, value in metadata.items():
        metadata_args.extend(["-metadata", f"{key}={value}"])

    video_name = video_path.name
    temp_output_file = Path(f'{video_path.parents[0]}/{video_name[:-4]}_temp.{video_name[-3:]}')

    # Use ffmpeg to add metadata
    !ffmpeg -i {video_path} -c copy -movflags use_metadata_tags -map_metadata 0 {" ".join(metadata_args)} {temp_output_file}

    # Remove the original file and rename the modified file
    os.remove(video_path)
    os.rename(temp_output_file, video_path)

## 💃 **FollowYourPose** 💃


In [21]:
# @title Config { display-mode: "form" }

class FYPConfig:
  # Init

  def __init__(self):

    # Const
    self.OPTIONS = [True, False]
    self.SAMPLE_YAML_PATH = Path(f"{CONFIG_DIR_PATH}/pose_sample.yaml")

    %cd -q {FYP_DIR_PATH}
    with open(self.SAMPLE_YAML_PATH, 'r') as yaml_file:
      load_config = yaml.load(yaml_file, Loader=yaml.FullLoader)
    self.load_config = load_config

    model_options = [("Default", Path(f"{FYP_DIR_PATH}/checkpoints/stable-diffusion-v1-4"))]
    for model_name in os.listdir(CUSTOM_MODEL_DIR_PATH):
        model_path = os.path.join(CUSTOM_MODEL_DIR_PATH, model_name)
        if os.path.isdir(model_path):
            model_options.append((model_name, Path(f"{CUSTOM_MODEL_DIR_PATH}/{model_name}")))
    value_model_path = Path(load_config['pretrained_model_path'])
    if value_model_path not in model_options:
      value_model_path = model_options[0][1]

    self.checkpoints_path = Path(load_config['resume_from_checkpoint'])
    if not os.path.isdir(self.checkpoints_path):
        self.checkpoints_path = Path(f"{CHECKPOINT_DIR_PATH}/{self.checkpoints_path.name}")

    # Create layout
    config_style = {'description_width': '150px'}
    config_layout = widgets.Layout(width="500px")
    config_button_layout = widgets.Layout(margin='0px 0px 20px 354px')

    # Inference Data
    self.pretrained_model_path = widgets.Dropdown(options=model_options,description="pretrained_model_path:",value=value_model_path,layout=config_layout,style=config_style)
    self.output_dir = widgets.Text(description="output_folder_name:", value="", style=config_style, layout=config_layout)
    self.train_batch_size = widgets.IntText(description="train_batch_size:", value=load_config['train_batch_size'], style=config_style, layout=config_layout)
    self.validation_steps = widgets.IntText(description="validation_steps:", value=load_config['validation_steps'], style=config_style, layout=config_layout)
    self.seed = widgets.IntText(description="seed:", value=load_config['seed'], style=config_style, layout=config_layout)
    self.mixed_precision = widgets.Text(description="mixed_precision:", value=load_config['mixed_precision'], style=config_style, layout=config_layout)
    self.gradient_checkpointing = widgets.Dropdown(options=self.OPTIONS, value=load_config['gradient_checkpointing'], description="gradient_checkpointing:", layout= config_layout, style=config_style)
    self.enable_xformers_memory_efficient_attention = widgets.Dropdown(options=self.OPTIONS, value=load_config['enable_xformers_memory_efficient_attention'], description="enable_xformers_memory_efficient_attention:", layout= config_layout, style=config_style)

    # Validation Data
    load_validation = load_config['validation_data']
    self.prompts = widgets.Textarea(description="prompts:", value="\n".join(load_validation['prompts']), style=config_style, layout=widgets.Layout(width="500px", height="100px"))
    self.video_length = widgets.IntText(description="video_length:", value=load_validation['video_length'], style=config_style, layout=config_layout)
    self.width = widgets.IntText(description="width:", value=load_validation['width'], style=config_style, layout=config_layout)
    self.height = widgets.IntText(description="height:", value=load_validation['height'], style=config_style, layout=config_layout)
    self.num_inference_steps = widgets.IntText(description="num_inference_steps:", value=load_validation['num_inference_steps'], style=config_style, layout=config_layout)
    self.guidance_scale = widgets.FloatText(description="guidance_scale:", value=load_validation['guidance_scale'], style=config_style, layout=config_layout)
    self.use_inv_latent = widgets.Dropdown(options=self.OPTIONS, value=load_validation['use_inv_latent'], description="use_inv_latent:", layout= config_layout, style=config_style)
    self.num_inv_steps = widgets.IntText(description="num_inv_steps:", value=load_validation['num_inv_steps'], style=config_style, layout=config_layout)
    self.num_dataset_set = widgets.Text(description="dataset_set:", value=load_validation['dataset_set'], style=config_style, layout=config_layout)

    # Subheader
    self.header_inference = widgets.HTML(value="<h3>Inference Data</h3>")
    self.header_validation = widgets.HTML(value="<h3>Validation Data</h3>")

  def getForm(self):
    config_vbox = widgets.VBox([
        self.header_inference,
        self.output_dir,
        self.pretrained_model_path,
        self.train_batch_size,
        self.validation_steps,
        self.seed,
        self.mixed_precision,
        self.gradient_checkpointing,
        self.enable_xformers_memory_efficient_attention,

        self.header_validation,
        self.prompts,
        self.video_length,
        self.width,
        self.height,
        self.num_inference_steps,
        self.guidance_scale,
        self.num_inv_steps,
        self.num_dataset_set
    ])

    return config_vbox

  # Create listeners
  def save_config(self):
    output_dir_path = Path(f"{INFERENCE_OUTPUT_DIR_PATH}/{self.output_dir.value}")
    if os.path.exists(output_dir_path):
      raise Exception("Duplicate output folder name")

    config = {
        "pretrained_model_path": self.pretrained_model_path.value.as_posix(),
        "output_dir": Path(f"{INFERENCE_OUTPUT_DIR_PATH}/{self.output_dir.value}").as_posix(),
        "validation_data": {
            "prompts": [prompt.strip() for prompt in self.prompts.value.splitlines() if prompt.strip()],
            "video_length": self.video_length.value,
            "width": self.width.value,
            "height": self.height.value,
            "num_inference_steps": self.num_inference_steps.value,
            "guidance_scale": self.guidance_scale.value,
            "use_inv_latent": self.use_inv_latent.value,
            "num_inv_steps": self.num_inv_steps.value,
            "dataset_set": self.num_dataset_set.value
        },
        "train_batch_size": self.train_batch_size.value,
        "validation_steps": self.validation_steps.value,
        "resume_from_checkpoint": self.checkpoints_path.as_posix(),
        "seed": self.seed.value,
        "mixed_precision": self.mixed_precision.value,
        "gradient_checkpointing": self.gradient_checkpointing.value,
        "enable_xformers_memory_efficient_attention": self.enable_xformers_memory_efficient_attention.value
    }
    with open(self.SAMPLE_YAML_PATH, "w") as file:
      yaml.dump(config, file, default_style='"', default_flow_style=False, sort_keys=False)


In [22]:
# @title Inference { display-mode: "form" }

def run_fyp_inference(config_file_path, video_file_path):
    %cd {FYP_DIR_PATH}

    # Start inference
    # For cross-platform compatibility, we do not run with flag TORCH_DISTRIBUTED_DEBUG=DETAIL
    !accelerate launch txt2video.py \
        --config={config_file_path}  \
        --skeleton_path={video_file_path}

In [23]:
# @title Post-process { display-mode: "form" }

def read_video_metadata(input_file):
    field_names = ["original_path", "bg_path"]

    cmd = f'ffprobe -v error -select_streams v:0 -show_entries "format_tags={",".join(field_names)}" -of json "{input_file}"'
    metadata_info = check_output(cmd, shell=True).decode()
    metadata_dict = json.loads(metadata_info)

    metadata = metadata_dict['format']['tags']
    return metadata["original_path"], metadata["bg_path"]

def postprocess_mmpose(skeleton_path, video_length, output_dir):

  # Get the video duration using ffprobe
  duration = float(check_output(['ffprobe', '-v', 'error', '-show_entries', 'format=duration', '-of', 'default=noprint_wrappers=1:nokey=1', skeleton_path]))

  # Set your desired duration
  desired_duration = video_length * 130 / 1000

  # Calculate the speedup factor
  speedup_factor = desired_duration / duration

  fps = video_length / desired_duration

  original_path, bg_path = read_video_metadata(skeleton_path)
  print(original_path, bg_path)

  # Run the FFmpeg command to adjust the video's duration
  output_pose_path = Path(f'{output_dir}/pose.gif')
  output_pose_bg_path = Path(f'{output_dir}/pose-bg.gif')
  output_pose_human_path = Path(f'{output_dir}/pose-human.gif')

  !ffmpeg -i $skeleton_path -vf "setpts=$speedup_factor*PTS,fps=$fps" $output_pose_path
  !ffmpeg -i $bg_path -vf "setpts=$speedup_factor*PTS,fps=$fps" $output_pose_bg_path
  !ffmpeg -i $original_path -vf "setpts=$speedup_factor*PTS,fps=$fps" $output_pose_human_path

def superimposeSkeleton(inf_folder_path):
  raw_path = Path(f"{inf_folder_path}/raw")
  skeleton = Path(f"{inf_folder_path}/pose.gif")

  output_folder_path = Path(f"{inf_folder_path}/superimposed")

  # Check if the superimposed folder exists
  if not os.path.exists(output_folder_path):
    # If it doesn't exist, create the superimposed folder
    os.makedirs(output_folder_path)

  # Get a list of all files in the directory
  humans = [f for f in os.listdir(raw_path) if os.path.isfile(os.path.join(raw_path, f))]

  for human_gif in humans:
    if not human_gif.endswith(".gif"):
      continue

    human_path = Path(f"{raw_path}/{human_gif}")
    output_file_path = Path(f"{output_folder_path}/{human_gif}")

    # Load your two GIFs
    fg = imageio.get_reader(skeleton)
    bg = imageio.get_reader(human_path)

    # Create a writer to save the result as a GIF
    output_gif = imageio.get_writer(output_file_path, fps=7.692, loop=0)  # Adjust the desired frame rate

    for i in range(min(len(fg), len(bg))):  # Process frames until one of the GIFs ends
        foreground = fg.get_data(i)
        background = bg.get_data(i)

        foreground = cv2.resize(foreground, (512, 512))
        background = cv2.resize(background, (512, 512))

        # Creating the alpha mask from the foreground image (e.g., removing the black background)
        gray = cv2.cvtColor(foreground, cv2.COLOR_BGR2GRAY)
        foreground = foreground.astype(float)
        background = background.astype(float)

        # Dark pixels filter (0 to 255)
        black_mask = (gray <= 50)

        # Combine the images based on the mask
        outImage = np.where(black_mask[:, :, np.newaxis], background, foreground)

        # Convert the frame to uint8
        ims = outImage.astype(np.uint8)

        # Add the frame to the output GIF
        output_gif.append_data(ims)

    output_gif.close()



## 🏋️ **Training** 🏋️

In [24]:
# @title Config { display-mode: "form" }

class TrainingConfig:
  # Init

  def __init__(self):

    # Const
    self.OPTIONS = [True, False]
    self.TRAIN_YAML_PATH = Path(f"{CONFIG_DIR_PATH}/pose_train.yaml")

    %cd -q {FYP_DIR_PATH}
    with open(self.TRAIN_YAML_PATH, 'r') as yaml_file:
      load_config = yaml.load(yaml_file, Loader=yaml.FullLoader)
    self.load_config = load_config

    model_options = [("Default", Path(f"{FYP_DIR_PATH}/checkpoints/stable-diffusion-v1-4"))]
    for model_name in os.listdir(CUSTOM_MODEL_DIR_PATH):
        model_path = os.path.join(CUSTOM_MODEL_DIR_PATH, model_name)
        if os.path.isdir(model_path):
            model_options.append((model_name, Path(f"{CUSTOM_MODEL_DIR_PATH}/{model_name}")))

    value_model_path = Path(load_config['pretrained_model_path'])
    if value_model_path not in model_options:
      value_model_path = model_options[0][1]

    # Create layout
    config_style = {'description_width': '150px'}
    config_layout = widgets.Layout(width="500px")
    config_button_layout = widgets.Layout(margin='0px 0px 20px 354px')

    # Basic Data
    self.pretrained_model_path = widgets.Dropdown(options=model_options,description="pretrained_model_path:",value=value_model_path,layout=config_layout,style=config_style)
    self.output_dir = widgets.Text(description="output_model_name:", value="", style=config_style, layout=config_layout)
    self.learning_rate = widgets.FloatText(description="learning_rate:", value=load_config['learning_rate'], style=config_style, layout=config_layout)
    self.train_batch_size = widgets.IntText(description="train_batch_size:", value=load_config['train_batch_size'], style=config_style, layout=config_layout)

    self.max_train_steps = widgets.IntText(description="max_train_steps:", value=load_config['max_train_steps'], style=config_style, layout=config_layout)
    self.seed = widgets.IntText(description="seed:", value=load_config['seed'], style=config_style, layout=config_layout)
    self.mixed_precision = widgets.Text(description="mixed_precision:", value=load_config['mixed_precision'], style=config_style, layout=config_layout)
    self.use_8bit_adam = widgets.Dropdown(options=self.OPTIONS, value=load_config['use_8bit_adam'], description="config_use_8bit_adam:", layout=config_layout, style=config_style)
    self.gradient_checkpointing = widgets.Dropdown(options=self.OPTIONS, value=load_config['gradient_checkpointing'], description="gradient_checkpointing:", layout=config_layout, style=config_style)
    self.enable_xformers_memory_efficient_attention = widgets.Dropdown(options=self.OPTIONS, value=load_config['enable_xformers_memory_efficient_attention'], description="enable_xformers_memory_efficient_attention:", layout=config_layout, style=config_style)

    # Train Data
    load_train = load_config['train_data']
    self.n_sample_frames = widgets.IntText(description="n_sample_frames:", value=load_train['n_sample_frames'], style=config_style, layout=config_layout)
    self.train_data_width = widgets.IntText(description="width:", value=load_train['width'], style=config_style, layout=config_layout)
    self.sample_frame_rate = widgets.IntText(description="sample_frame_rate:", value=load_train['sample_frame_rate'], style=config_style, layout=config_layout)

    # Subheader
    self.header_generic = widgets.HTML(value="<h3>Generic Data</h3>")
    self.header_training = widgets.HTML(value="<h3>Training Data</h3>")

  def getForm(self):
    config_vbox = widgets.VBox([
        self.header_generic,
        self.output_dir,
        self.pretrained_model_path,
        self.train_batch_size,
        self.max_train_steps,
        self.seed,
        self.mixed_precision,
        self.use_8bit_adam,
        self.gradient_checkpointing,
        self.enable_xformers_memory_efficient_attention,

        self.header_training,
        self.n_sample_frames,
        self.train_data_width,
        self.sample_frame_rate
    ])

    return config_vbox

  # Create listeners
  def save_config(self, video_path):
    config = {
        "pretrained_model_path": Path(self.pretrained_model_path.value).as_posix(),
        "output_dir": Path(f"{CUSTOM_MODEL_DIR_PATH}/{self.output_dir.value}").as_posix(),
        "train_data": {
            "video_path": Path(video_path).as_posix(),
            "n_sample_frames": self.n_sample_frames.value,
            "width": self.train_data_width.value,
            "sample_frame_rate": self.sample_frame_rate.value
        },
        "learning_rate": self.learning_rate.value,
        "train_batch_size": self.train_batch_size.value,
        "max_train_steps": self.max_train_steps.value,
        "seed": self.seed.value,
        "mixed_precision": self.mixed_precision.value,
        "use_8bit_adam": self.use_8bit_adam.value,
        "gradient_checkpointing": self.gradient_checkpointing.value,
        "enable_xformers_memory_efficient_attention": self.enable_xformers_memory_efficient_attention.value
    }
    with open(self.TRAIN_YAML_PATH, "w") as file:
      yaml.dump(config, file, default_style='"', default_flow_style=False, sort_keys=False)


In [25]:
# @title Video Cutter { display-mode: "form" }

def cut_video(training_dataset, training_branch):
  #Loop video files from selected dataset folder
  for video_file in os.listdir(training_dataset):
    video, ext = os.path.splitext(video_file)

    # Ignore non video files (Eg: .ipynb_checkpoint and csv)
    if ext not in ALLOWED_VIDEO_EXT:
      continue

    video_folder = Path(f"{training_branch}/{video}")
    if not os.path.exists(video_folder):
      os.mkdir(video_folder)

      # Retrieve charade object by ID
      charade_data = None
      for charade in charades_all:
        if charade.id == video:
            charade_data = charade
            break

      if not charade_data:
        sims_data = SimsData(video)
        if sims_data.isSims:
          print(f"{video}: Non-charades clip, no clipping required")
          input_video = Path(f"{training_dataset}/{video_file}")
          output_video = Path(f"{video_folder}/{video}01{ext}")
          !cp {input_video} {output_video}
        else:
          os.rmdir(video_folder)
          print(f"{video}: Not found in charades or sims, skip clipping")
      elif not charade_data.actions:    # If no clipping required, keep whole video
        print(f"No clipping needed for {video}")
        input_video = Path(f"{training_dataset}/{video_file}")
        output_video = Path(f"{video_folder}/{video}01{ext}")
        !cp {input_video} {output_video}
      else:
        print(f"Clipping {video}")
        charade_actions = charade_data.actions.items()
        total_charade_actions = len(charade_actions)
        for i, (class_id, timings) in enumerate(charade_actions):

          input_video = Path(f"{training_dataset}/{video_file}")
          output_video = Path(f"{video_folder}/{video}{i+1:02}{ext}")

          print(f"#{i+1}/{total_charade_actions}: {timings[0]}ms to {timings[0]+timings[1]}ms [I:{input_video}] [O:{output_video}]")
          !ffmpeg -i {input_video} -ss {timings[0]}ms -t {timings[1]}ms -c:v libx264 -c:a aac {output_video} -loglevel quiet
    else:
      print(f"Folder already exist for video_id: {video}. Skipping ...")

In [26]:
# @title Generate Metadata { display-mode: "form" }

def generate_training_metadata(training_branch):
  training_metadata_file = Path(f"{training_branch}/metadata.tsv")
  with open(training_metadata_file, 'w', newline='', encoding='utf-8') as tsvfile:
    fieldnames = ['part_id', 'clip_id', 'caption']
    writer = csv.DictWriter(tsvfile, fieldnames=fieldnames, delimiter='\t')
    writer.writeheader()

    for part_id in os.listdir(training_branch):
      folder_path = os.path.join(training_branch, part_id)

      # Ignore non video files (Eg: .ipynb_checkpoint and csv)
      if not os.path.isdir(folder_path) or part_id.startswith("."):
        continue

      charade_data = None
      for charade in charades_all:
        if charade.id == part_id:
            charade_data = charade
            break

      if not charade_data:
        sims_data = SimsData(part_id)
        if not sims_data.isSims:
          print("Missing charades data, skipping ...")
          continue

      # Sort by video sub-id to maintain order
      training_video_files = sorted(os.listdir(folder_path), key=lambda x: int(os.path.splitext(x)[0][-2:]))
      for i, clip in enumerate(training_video_files):
        caption = charade_data.getCaption(i) if charade_data else sims_data.getCaption()
        writer.writerow({
            'part_id': part_id,
            'clip_id': clip,
            'caption': caption
        })
  print(f"TSV created: {training_metadata_file}")

In [27]:
# @title Execution { display-mode: "form" }

# Changed fields in pose_train
## output_dir: "/content/custom_model/<model_name>" [test01]
## video_path: "/content/training_content/<dataset_folder>" [20231007_161145]

def run_model_training():
  %cd {FYP_DIR_PATH}

  # For cross-platform compatibility, we do not run with flag TORCH_DISTRIBUTED_DEBUG=DETAIL
  !accelerate launch train_followyourpose.py --config="configs/pose_train.yaml"

## 🧪 **Testing** 🧪


In [28]:
# @title Config { display-mode: "form" }

class TestConfig:
  # Init

  def __init__(self):

    # Const
    self.OPTIONS = [True, False]
    self.TEST_YAML_PATH = Path(f"{CONFIG_DIR_PATH}/pose_test.yaml")

    %cd -q {FYP_DIR_PATH}
    with open(self.TEST_YAML_PATH, 'r') as yaml_file:
      load_config = yaml.load(yaml_file, Loader=yaml.FullLoader)
    self.load_config = load_config

    model_options = [("Default", Path(f"{FYP_DIR_PATH}/checkpoints/stable-diffusion-v1-4"))]
    for model_name in os.listdir(CUSTOM_MODEL_DIR_PATH):
        model_path = os.path.join(CUSTOM_MODEL_DIR_PATH, model_name)
        if os.path.isdir(model_path):
            model_options.append((model_name, Path(f"{CUSTOM_MODEL_DIR_PATH}/{model_name}")))

    value_model_path = Path(load_config['pretrained_model_path'])
    if value_model_path not in model_options:
      value_model_path = model_options[0][1]

    self.checkpoints_path = Path(load_config['resume_from_checkpoint'])
    if not os.path.isdir(self.checkpoints_path):
        self.checkpoints_path = Path(f"{CHECKPOINT_DIR_PATH}/{self.checkpoints_path.name}")

    # Create layout
    config_style = {'description_width': '150px'}
    config_layout = widgets.Layout(width="500px")
    config_button_layout = widgets.Layout(margin='0px 0px 20px 354px')

    # Basic Data
    self.pretrained_model_path = widgets.Dropdown(options=model_options,description="pretrained_model_path:",value=value_model_path,layout=config_layout,style=config_style)

    # Validation Data
    load_validation = load_config['validation_data']
    self.num_inference_steps = widgets.IntText(description="num_inference_steps:", value=load_validation['num_inference_steps'], style=config_style, layout=config_layout)
    self.prompts = widgets.Textarea(description="prompts:", value="\n".join(load_validation['prompts']), style=config_style, layout=widgets.Layout(width="500px", height="100px"))
    self.video_length = widgets.IntText(description="video_length:", value=load_validation['video_length'], style=config_style, layout=config_layout)

    # Subheader
    self.header_generic = widgets.HTML(value="<h3>Generic Data</h3>")
    self.header_validation = widgets.HTML(value="<h3>Validation Data</h3>")

  def getForm(self):
    config_vbox = widgets.VBox([
        self.header_generic,
        self.pretrained_model_path,

        self.header_validation,
        self.num_inference_steps,
        self.prompts,
        self.video_length,
    ])

    return config_vbox

  # Create listeners
  def save_config(self):

    self.load_config["pretrained_model_path"] = self.pretrained_model_path.value.as_posix()
    self.load_config["validation_data"]["prompts"] = [prompt.strip() for prompt in self.prompts.value.splitlines() if prompt.strip()]
    self.load_config["validation_data"]["video_length"] = self.video_length.value
    self.load_config["validation_data"]["num_inference_steps"] = self.num_inference_steps.value
    self.load_config["resume_from_checkpoint"] = self.checkpoints_path.as_posix()

    with open(self.TEST_YAML_PATH, "w") as file:
      yaml.dump(self.load_config, file, default_style='"', default_flow_style=False, sort_keys=False)

  def update_output_dir(self, output_dir_folder_name):
    self.load_config["output_dir"] = output_dir_folder_name

    print(self.load_config["output_dir"])
    with open(self.TEST_YAML_PATH, "w") as file:
      yaml.dump(self.load_config, file, default_style='"', default_flow_style=False, sort_keys=False)

  def get_output_dir(self):
    return self.load_config["output_dir"]

In [29]:
# @title Convert Gif to Mp4 { display-mode: "form" }

def convertGifs(inf_path):
  raw_gifs = [f for f in os.listdir(f"{inf_path}/raw") if os.path.isfile(os.path.join(f"{inf_path}/raw", f))]

  for gif in raw_gifs:
    # Load the GIF using VideoFileClip
    gif_path = Path(f"{inf_path}/raw/{gif}")
    gif_clip = VideoFileClip(gif_path.as_posix())

    os.makedirs(f"{inf_path}/gif_to_video", exist_ok=True)

    # Define the output MP4 file name
    mp4_path =  Path(f"{inf_path}/gif_to_video/{gif[:-4]}.mp4")

    # Write the GIF as an MP4 video
    gif_clip.write_videofile(mp4_path.as_posix(), codec='libx264')

  # Convert the pose human gif to mp4 for testing
  human_gif_path = Path(f"{inf_path}/pose-human.gif")
  human_gif_clip = VideoFileClip(human_gif_path.as_posix())

  # Define the output MP4 file name
  human_mp4_path =  Path(f"{inf_path}/gif_to_video/pose-human.mp4")

  # Write the GIF as an MP4 video
  human_gif_clip.write_videofile(human_mp4_path.as_posix(), codec='libx264')

In [30]:
# @title Score Function { display-mode: "form" }

def create_or_append_csv(filename, data):
  # Check if the file exists
  file_exists = os.path.isfile(filename)
  csv_header = ["Pose", "Prompt", "Pose Score", "Environment Score"]

  with open(filename, 'a', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)

    # If the file doesn't exist, write the header row
    if not file_exists and csv_header:
        csv_writer.writerow(csv_header)

    # Write the data
    csv_writer.writerow(data)

def calculate_score(skeleton1, skeleton2):
  all_keypoints1 = []
  all_keypoints2 = []

  for frame in skeleton1:
    for instance in frame["instances"]:
      keypoints = instance["keypoints"]
      all_keypoints1.append(keypoints)

  for frame in skeleton2:
    for instance in frame["instances"]:
      keypoints = instance["keypoints"]
      all_keypoints2.append(keypoints)

  # Perform keypoint-based similarity measurement (e.g., using Euclidean distance)
  def euclidean_distance(p1, p2):
      return np.linalg.norm(np.array(p1) - np.array(p2))

  # Calculate the similarity score based on keypoint positions
  similarity_scores_per_frame = []
  similarity_scores = []
  keep_count_outer = 1
  for keypoints1, keypoints2 in zip(all_keypoints1, all_keypoints2):
    for point1, point2 in zip(keypoints1, keypoints2):

      filtered_keypoints1 = []
      filtered_keypoints2 = []

      # Remove all keypoints <= 0 as values < 0 are out of bounds (if origin is top left)
      if point1[0] > 0 and point2[0] > 0:
        filtered_keypoints1.append(point1)
        filtered_keypoints2.append(point2)

      distance = euclidean_distance(filtered_keypoints1, filtered_keypoints2)
      similarity_scores_per_frame.append(distance)

    per_frame_similarity = sum(similarity_scores_per_frame) / len(similarity_scores_per_frame)
    similarity_scores.append(per_frame_similarity)
    # print(f"Similarity Score at frame {keep_count_outer}: {per_frame_similarity / 10}")
    keep_count_outer += 1

  # Calculate an overall similarity score (e.g., average or sum of distances)
  # Scores from 0 to 100 with 0 being identical and 100 being the entire worlds apart
  overall_similarity = (sum(similarity_scores) / len(similarity_scores)) / 10

  return overall_similarity


## 🧮 **Classes** 🧮

### **Initialise**
* CSS
* Enums
* Abstract Classes


In [31]:
#@title CSS
svg_loading = """<svg class="ring" viewBox="25 25 50 50" stroke-width="8"><circle cx="50" cy="50" r="20" /></svg>"""
svg_done = """<svg width="25px" class="check" viewBox="0 0 16 16"><path stroke-width="10" d="M8 16A8 8 0 1 1 8 0a8 8 0 0 1 0 16Zm3.78-9.72a.751.751 0 0 0-.018-1.042.751.751 0 0 0-1.042-.018L6.75 9.19 5.28 7.72a.751.751 0 0 0-1.042.018.751.751 0 0 0-.018 1.042l2 2a.75.75 0 0 0 1.06 0Z" /></svg>"""
css = """<style>.check {background-color: white;border-radius: 100%;fill: green;margin: 2.5px 6px 2.5px 0px;}.ring {--uib-size: 25px;--uib-speed: 2s;--uib-color: gray;height: var(--uib-size);width: var(--uib-size);vertical-align: middle;transform-origin: center;animation: rotate var(--uib-speed) linear infinite;margin: 2.5px 6px 2.5px 0px;}.ring circle {fill: none;stroke: var(--uib-color);stroke-dasharray: 1, 200;stroke-dashoffset: 0;stroke-linecap: round;animation: stretch calc(var(--uib-speed) * 0.75) ease-in-out infinite;}@keyframes rotate {100% {transform: rotate(360deg);}}@keyframes stretch {0% {stroke-dasharray: 1, 200;stroke-dashoffset: 0;}50% {stroke-dasharray: 90, 200;stroke-dashoffset: -35px;}100% {stroke-dashoffset: -124px;}}</style>"""


In [32]:
#@title Enums
class State(Enum):
  INTRO = 1
  SETUP = 2
  EXPLORE = 3
  MMPOSE = 4
  FYP = 5
  TRAIN = 6
  TEST = 7

In [33]:
#@title BaseState
class BaseState:
  def __init__(self, state_manager, title):
    self.title = title
    self.state_manager = state_manager
  def getScreen(self):
    raise NotImplementedError()

### **State Manager**


In [34]:
#@title Statemanager
class StateManager:
  def __init__(self):
    self.state = State.INTRO
    self.state_classes = {
        State.INTRO: IntroState,
        State.SETUP: SetupState,
        State.EXPLORE: ExplorationState,
        State.MMPOSE: MMPoseState,
        State.FYP: FYPState,
        State.TRAIN: TrainState,
        State.TEST: TestState
    }
    # self.state_object = self.state_classes[self.state]
    self.screen = widgets.Output()

    self.state_object = self.state_classes[self.state](self)
    display(widgets.HTML(css))


  def start(self):
    display(self.screen)
    self.display_screen()

  def display_screen(self):
    with self.screen:
      clear_output()
      display(self.state_object.getScreen())

  def clear_console(self):
    clear_output(wait=True)
    display(widgets.HTML(css))
    display(self.screen)

  def change_state(self, new_state, content=None):
    self.state = new_state
    if content:
      self.state_object = self.state_classes[self.state](self, content=content)
    else:
      self.state_object = self.state_classes[self.state](self)
    self.clear_console()
    self.display_screen()

### **States**
* Intro
* Setup
* Explore
* MMPose
* FYP
* Train

In [35]:
#@title IntroState
class IntroState(BaseState):
  def __init__(self, state_manager):
    BaseState.__init__(self, state_manager, "Welcome")
    self.subtitle = widgets.HTML("""
      <b>A Jupyter Notebook Wrapper that combines the works of FollowYourPose & MMPose.</b><br>
      Brought to you by team 05:
      <ul>
        <li>Aloysius</li>
        <li>Max</li>
        <li>Zhenglong</li>
        <li>Ai Xin</li>
        <li>Huai Fu</li>
      </ul>
    """)
    self.get_started_button = widgets.Button(description="Get Started >>")
    self.get_started_button.on_click(lambda x: self.state_manager.change_state(State.SETUP))

    self.screen = widgets.AppLayout(
        header=widgets.HTML(value=f"<h1>{self.title}</h1>"),
        left_sidebar=None,
        center=self.getContent(),
        right_sidebar=None,
        footer=None,
        pane_heights=[1,5,1]
    )

  def getScreen(self):
    return self.screen

  def getContent(self):
    vbox = widgets.VBox([self.subtitle, self.get_started_button])
    return vbox

In [36]:
#@title SetupState
class SetupState(BaseState):
  def __init__(self, state_manager):

    self.isColab = bool(os.getenv("COLAB_RELEASE_TAG"))
    BaseState.__init__(self, state_manager, "Setup")
    self.platform = sys.platform

    ram_bytes = psutil.virtual_memory().available
    ram_gb = ram_bytes / (1024 ** 3)
    self.avail_ram = f"{ram_gb:.2f}"

    # Button
    self.skip_button = widgets.Button(description="Skip >>")
    self.skip_button.on_click(self.skip_check)

    self.mount_button = widgets.Button(description="Mount", button_style="info")
    self.mount_button.on_click(lambda x: self.run_mount_account_select())

    self.install_checkpoints_button = widgets.Button(description="Install", button_style="info")
    self.install_checkpoints_button.on_click(lambda x: self.run_windows_setup())

    self.continue_button = widgets.Button(description="Continue", button_style="info")
    self.continue_button.on_click(lambda x: self.run_colab_setup())

    self.done_button = widgets.Button(description="Done", button_style="success")
    self.done_button.on_click(lambda x: self.state_manager.change_state(State.EXPLORE))

    self.install_button = widgets.Button(description="Install", button_style="info")
    self.install_button.on_click(lambda x: self.run_windows_setup())

    # Substates
    self.stage = 1
    self.setup_stage = [False, False, False, False, False, False, False, False, False]

    self.screen = widgets.AppLayout(
        header=widgets.HTML(value=f"<h1>{self.title}</h1>"),
        left_sidebar=None,
        center=None,
        right_sidebar=None,
        footer=None,
        pane_heights=[1,10,1]
      )

    self.build_screen()

  def getScreen(self):
    return self.screen

  def colab_screen(self):
    hr = widgets.HTML("<hr>", layout=widgets.Layout(width="300px"))
    label_platform = widgets.HTML(f"""
      <b>Current Platform: </b>{self.platform}<br>
      <b>Available Ram: </b>{self.avail_ram}GB<br>
      <b>Est Setup Time: </b>8 mins
    """)

    vbox_items = []
    if self.stage <= 3:
      label_mount = widgets.HTML("""
          <h2>Account login for drive mounting</h2>
          Please make sure that your google drive contains a <b>checkpoints</b> at your root!
          <ol>
            <li>Click mount to start then wait for console output</li>
            <li>Click on the URL of the Failure in the output Failure("Error opening URL: ...")</li>
            <li>Login to the appropriate account</li>
            <li>Once successful, you can close the tab</li>
            <li><b>Click continue only when you have login into the correct account</b></li>
          </ol>
        """)

      if self.stage == 1:
        hbox_button = widgets.HBox([self.mount_button, self.skip_button])
        vbox_items = [label_platform, hr, label_mount, hbox_button]
      elif self.stage == 2:
        vbox_items = [label_platform, hr, label_mount]
      else:
        vbox_items = [label_platform, hr, label_mount, self.continue_button]

    elif self.stage >= 4:

      setup_label_1 = widgets.HBox([widgets.HTML(svg_done if self.setup_stage[0] else svg_loading),widgets.HTML("<b>Setting up python environment</b>")])
      setup_label_2 = widgets.HBox([widgets.HTML(svg_done if self.setup_stage[1] else svg_loading),widgets.HTML("<b>Cloning git repository</b>")])
      setup_label_3 = widgets.HBox([widgets.HTML(svg_done if self.setup_stage[2] else svg_loading),widgets.HTML("<b>Loading default YAMLs</b>")])
      setup_label_4 = widgets.HBox([widgets.HTML(svg_done if self.setup_stage[3] else svg_loading),widgets.HTML("<b>Downloading dependency libraries</b>")])
      setup_label_5 = widgets.HBox([widgets.HTML(svg_done if self.setup_stage[4] else svg_loading),widgets.HTML("<b>Mounting drive contents</b>")])
      setup_label_6 = widgets.HBox([widgets.HTML(svg_done if self.setup_stage[5] else svg_loading),widgets.HTML("<b>Creating directories</b>")])
      setup_label_7 = widgets.HBox([widgets.HTML(svg_done if self.setup_stage[6] else svg_loading),widgets.HTML("<b>Linking symbolic paths</b>")])
      setup_label_8 = widgets.HBox([widgets.HTML(svg_done if self.setup_stage[7] else svg_loading),widgets.HTML("<b>Loading charades metadata</b>")])
      setup_label_9 = widgets.HBox([widgets.HTML(svg_done if self.setup_stage[8] else svg_loading),widgets.HTML("<b>Loading sims metadata</b>")])

      vbox_items = [label_platform, hr, setup_label_1, setup_label_2, setup_label_3, setup_label_4, setup_label_5, setup_label_6, setup_label_7, setup_label_8, setup_label_9, hr]
      if self.stage == 5:
        vbox_items.append(self.done_button)

    self.screen.center = widgets.VBox(vbox_items, layout=widgets.Layout(margin="0px 0px 30px 0px"))

  def windows_screen(self):
    hr = widgets.HTML("<hr>", layout=widgets.Layout(width="300px"))
    label_platform = widgets.HTML(f"""
      <b>Current Platform: </b>{self.platform}<br>
      <b>Available Ram: </b>{self.avail_ram}GB<br>
      <b>Est Setup Time: </b>8 mins
    """)

    vbox_items = []
    if self.stage == 1:
      label_desc = widgets.HTML("""
        <h2>Local (Windows) Setup</h2>
        You only need to run this cell once after cloning. Subsequent launches do not require running of these cells.
        To prevent outputs from being truncated in the local version of Jupyter Notebook, press [Shift-O].
      """)
      hbox_button = widgets.HBox([self.install_button, self.skip_button])
      vbox_items = [label_platform, hr, label_desc, hbox_button]

    elif self.stage >= 2:

      setup_label_1
      setup_label_1 = widgets.HBox([widgets.HTML(svg_done if self.setup_stage[0] else svg_loading),widgets.HTML("<b>Cloning git repository</b>")])
      setup_label_2 = widgets.HBox([widgets.HTML(svg_done if self.setup_stage[1] else svg_loading),widgets.HTML("<b>Loading default YAMLs</b>")])
      setup_label_3 = widgets.HBox([widgets.HTML(svg_done if self.setup_stage[2] else svg_loading),widgets.HTML("<b>Creating directories</b>")])
      setup_label_4 = widgets.HBox([widgets.HTML(svg_done if self.setup_stage[3] else svg_loading),widgets.HTML("<b>Loading charades metadata</b>")])
      setup_label_5 = widgets.HBox([widgets.HTML(svg_done if self.setup_stage[4] else svg_loading),widgets.HTML("<b>Loading sims metadata</b>")])

      vbox_items = [label_platform, hr, setup_label_1, setup_label_2, setup_label_3, setup_label_4, setup_label_5, hr]
      if self.stage == 3:
        vbox_items.append(self.done_button)

    self.screen.center = widgets.VBox(vbox_items, layout=widgets.Layout(margin="0px 0px 30px 0px"))

  def build_screen(self):
    if self.isColab:
      self.colab_screen()
    else:
      self.windows_screen()

  def skip_check(self, change):
    if not sims_const:
      load_sims()
    if not action_descriptions or not charades_all:
      load_charades()
    self.state_manager.change_state(State.EXPLORE)

  def run_windows_setup(self):
    self.stage = 2
    self.build_screen()

    setup_github(self.isColab)
    self.setup_stage[0] = True
    self.build_screen()

    setup_default_yamls(self.isColab)
    self.setup_stage[1] = True
    self.build_screen()

    create_dir_path()
    self.setup_stage[2] = True
    self.build_screen()

    load_charades()
    self.setup_stage[3] = True
    self.build_screen()

    load_sims()
    self.setup_stage[4] = True
    self.build_screen()

    self.stage = 3
    self.build_screen()

  def run_colab_setup(self):
    self.stage = 4
    self.build_screen()

    setup_python()
    self.setup_stage[0] = True
    self.build_screen()

    setup_github(self.isColab)
    self.setup_stage[1] = True
    self.build_screen()

    setup_default_yamls(self.isColab)
    self.setup_stage[2] = True
    self.build_screen()

    setup_dependency_lib()
    self.setup_stage[3] = True
    self.build_screen()

    mount_drive()
    self.setup_stage[4] = True
    self.build_screen()

    create_dir_path()
    self.setup_stage[5] = True
    self.build_screen()

    symbolic_linking()
    self.setup_stage[6] = True
    self.build_screen()

    load_charades()
    self.setup_stage[7] = True
    self.build_screen()

    load_sims()
    self.setup_stage[8] = True
    self.build_screen()

    self.stage = 5
    self.build_screen()


  def run_mount_account_select(self):
    self.stage = 2
    self.build_screen()

    mount_account_select()
    self.stage = 3
    self.build_screen()



In [37]:
#@title ExplorationState

class ExplorationState(BaseState):
  def __init__(self, state_manager):
    BaseState.__init__(self, state_manager, "Data Exploration")

    self.video_output = widgets.Output()
    self.video_tab = VideoTab(self)

    self.gif_output = widgets.Output()
    self.gif_tab = GifTab(self)

    self.grade_output = widgets.Output()
    self.grade_tab = GradeTab(self)

    self.chart_output = widgets.Output()
    self.chart_tab = ChartTab(self)

    self.video_tab.attach_listeners()
    self.video_tab.display_content()

    self.gif_tab.attach_listeners()
    self.gif_tab.display_content()

    self.grade_tab.attach_listeners()
    self.grade_tab.display_content()

    self.chart_tab.attach_listeners()
    self.chart_tab.display_content()

    tab = widgets.Tab(layout=widgets.Layout(width="1500px"))
    tab.children = [self.video_output, self.gif_output, self.grade_output, self.chart_output]
    tab.set_title(0, "Video")
    tab.set_title(1, "GIF")
    tab.set_title(2, "Test Grade")
    tab.set_title(3, "Chart")

    self.screen = widgets.AppLayout(
        header=widgets.HTML(value=f"<h1>{self.title}</h1>"),
        left_sidebar=None,
        center=tab,
        right_sidebar=None,
        footer=widgets.HTML("<style>.jupyter-widgets.widget-tab > .p-TabBar .p-TabBar-tabLabel{color: white; font-weight: bold;}</style>"),
        pane_heights=[1,10,1]
    )

  def getScreen(self):
    return self.screen

  def getContent(self):
    vbox = widgets.HTML(value=f"<h1>{self.title}</h1>") # widgets.VBox([self.subtitle, self.get_started_button])
    return vbox

  def change_state(self, new_state, content):
    self.state_manager.change_state(new_state, content)


In [38]:
#@title MMPoseState

class MMPoseState(BaseState):
  def __init__(self, state_manager, content):
    BaseState.__init__(self, state_manager, "MMPose")
    self.content = content
    self.screen = widgets.AppLayout(
      header=widgets.HTML(value=f"<h1>{self.title}</h1>"),
      left_sidebar=None,
      center=None,
      right_sidebar=None,
      footer=None,
      pane_heights=[1,5,1]
    )
    self.state = 1
    self.mmpose_stage = [False, False, False, False]

    self.back_button = widgets.Button(description="Back")
    self.back_button.on_click(lambda x: self.state_manager.change_state(State.EXPLORE))

    self.start_button = widgets.Button(description="Start", button_style="info")
    self.start_button.on_click(lambda x: self.run_mmpose())

    self.done_button = widgets.Button(description="Done", button_style="info")
    self.done_button.on_click(lambda x: self.state_manager.change_state(State.EXPLORE))

    self.build_screen()

  def getScreen(self):
    return self.screen

  def build_screen(self):
    label_video = widgets.HTML(f"""
      <b>Selected Video: </b>{self.content[1]}<br>
    """)

    vbox_content = []
    if self.state == 1:
      hbox_button = widgets.HBox([self.start_button, self.back_button])
      vbox_content = [label_video, hbox_button]
    elif self.state >= 2:
      hr = widgets.HTML("<hr>", layout=widgets.Layout(width="300px"))
      mmpose_label_1 = widgets.HBox([widgets.HTML(svg_done if self.mmpose_stage[0] else svg_loading),widgets.HTML("<b>Converting video to skeleton with background</b>")])
      mmpose_label_2 = widgets.HBox([widgets.HTML(svg_done if self.mmpose_stage[1] else svg_loading),widgets.HTML("<b>Converting video to skeleton</b>")])
      mmpose_label_3 = widgets.HBox([widgets.HTML(svg_done if self.mmpose_stage[2] else svg_loading),widgets.HTML("<b>Re-encoding videos</b>")])
      mmpose_label_4 = widgets.HBox([widgets.HTML(svg_done if self.mmpose_stage[3] else svg_loading),widgets.HTML("<b>Adding metadata</b>")])

      vbox_content = [label_video, hr, mmpose_label_1, mmpose_label_2, mmpose_label_3, hr]

      if self.state == 3:
        vbox_content.append(self.done_button)

    self.screen.center = widgets.VBox(vbox_content)

  def run_mmpose(self):
    self.state = 2
    self.build_screen()

    folder_path, video_path = self.content
    full_path = Path(f"{VIDEO_DIR_PATH}/{folder_path}/{video_path}")
    bg_file = Path(f"{VIDEO_SKELETON_DIR_PATH}/.{video_path[:-4]}-bg.{video_path[-3:]}")
    metadata = {
        "original_path": full_path.as_posix(),
        "bg_path": bg_file.as_posix()
    }

    skeleton_output_with_bg_path = run_mmpose_with_bg_inference(folder_path, video_path)
    self.mmpose_stage[0] = True
    self.build_screen()

    skeleton_output_path = run_mmpose_inference(folder_path, video_path)
    self.mmpose_stage[1] = True
    self.build_screen()

    reencode_video(skeleton_output_path)
    reencode_video(skeleton_output_with_bg_path)
    self.mmpose_stage[2] = True
    self.build_screen()

    add_metadata_to_mp4(skeleton_output_path, metadata)
    self.mmpose_stage[3] = True
    self.build_screen()

    self.state = 3
    self.build_screen()




In [39]:
#@title FYPState

class FYPState(BaseState):
  def __init__(self, state_manager, content):
    BaseState.__init__(self, state_manager, "FollowYourPose")
    self.content = content
    self.screen = widgets.AppLayout(
      header=widgets.HTML(value=f"<h1>{self.title}</h1>"),
      left_sidebar=None,
      center=None,
      right_sidebar=None,
      footer=None,
      pane_heights=[1,10,1]
    )
    self.stage = 1
    self.fyp_stage = [False, False, False, False]

    self.back_button = widgets.Button(description="Back")
    self.back_button.on_click(lambda x: self.state_manager.change_state(State.EXPLORE))

    self.start_button = widgets.Button(description="Start", button_style="info")
    self.start_button.on_click(lambda x: self.run_fyp())

    self.done_button = widgets.Button(description="Done", button_style="info")
    self.done_button.on_click(lambda x: self.state_manager.change_state(State.EXPLORE))

    self.FYPConfig = FYPConfig()
    self.build_screen()

  def getScreen(self):
    return self.screen

  def build_screen(self):
    label_video = widgets.HTML(f"""
      <b>Selected Skeleton Video: </b>{self.content[1]}<br>
    """)
    hr = widgets.HTML("<hr>", layout=widgets.Layout(width="300px"))

    vbox_content = []
    if self.stage <= 2:
      hbox_btn = widgets.HBox([self.start_button, self.back_button])
      vbox_content = [label_video, self.FYPConfig.getForm(), hr, hbox_btn]
      if self.stage == 2:
        label_error = widgets.HTML("<font color='red'>Duplicate output folder name")
        vbox_content.append(label_error)
    elif self.stage >= 3:
      fyp_label_1 = widgets.HBox([widgets.HTML(svg_done if self.fyp_stage[0] else svg_loading),widgets.HTML("<b>Saving configs</b>")])
      fyp_label_2 = widgets.HBox([widgets.HTML(svg_done if self.fyp_stage[1] else svg_loading),widgets.HTML("<b>Running inference</b>")])
      fyp_label_3 = widgets.HBox([widgets.HTML(svg_done if self.fyp_stage[2] else svg_loading),widgets.HTML("<b>Processing pose gif</b>")])
      fyp_label_4 = widgets.HBox([widgets.HTML(svg_done if self.fyp_stage[3] else svg_loading),widgets.HTML("<b>Generating combined gif</b>")])

      vbox_content = [label_video, hr, fyp_label_1, fyp_label_2, fyp_label_3, fyp_label_4, hr]

      if self.stage == 4:
        label_folder_output = widgets.HTML(f"<b>Output Folder: </b>{self.FYPConfig.output_dir.value}<br>")

        vbox_content.extend([label_folder_output, self.done_button])

    self.screen.center = widgets.VBox(vbox_content)

  def run_fyp(self):
    skeleton_path = Path(f"{VIDEO_DIR_PATH}/{self.content[0]}/{self.content[1]}")
    inf_path = Path(f"{INFERENCE_OUTPUT_DIR_PATH}/{self.FYPConfig.output_dir.value}")

    self.stage = 3
    self.build_screen()

    try:
      self.FYPConfig.save_config()
      self.fyp_stage[0] = True
      self.build_screen()
    except:
      self.stage = 2
      self.build_screen()
      return

    run_fyp_inference(Path(f"{CONFIG_DIR_PATH}/pose_sample.yaml"), skeleton_path)
    self.fyp_stage[1] = True
    self.build_screen()

    postprocess_mmpose(skeleton_path, self.FYPConfig.video_length.value, inf_path)
    self.fyp_stage[2] = True
    self.build_screen()

    superimposeSkeleton(inf_path)
    self.fyp_stage[3] = True
    self.build_screen()

    self.stage = 4
    self.build_screen()


In [40]:
# @title TrainState { display-mode: "form" }

class TrainState(BaseState):
  def __init__(self, state_manager, content):
    BaseState.__init__(self, state_manager, "Train Model")
    self.content = content
    self.total_videos = self.total_folder_items()

    self.training_idx = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    self.training_branch = Path(f"{TRAINING_CONTENT_DIR_PATH}/{self.training_idx}")
    self.training_dataset = Path(f"{VIDEO_DIR_PATH}/{self.content}")

    self.screen = widgets.AppLayout(
      header=widgets.HTML(value=f"<h1>{self.title}</h1>"),
      left_sidebar=None,
      center=None,
      right_sidebar=None,
      footer=None,
      pane_heights=[1,10,1]
    )
    self.stage = 1
    self.train_stage = [False, False, False, False]

    self.back_button = widgets.Button(description="Back")
    self.back_button.on_click(lambda x: self.state_manager.change_state(State.EXPLORE))

    self.start_button = widgets.Button(description="Start", button_style="info")
    self.start_button.on_click(lambda x: self.run_train())

    self.done_button = widgets.Button(description="Done", button_style="info")
    self.done_button.on_click(lambda x: self.state_manager.change_state(State.EXPLORE))

    self.TrainingConfig = TrainingConfig()
    self.build_screen()

  def getScreen(self):
    return self.screen

  def build_screen(self):
    label_folder = widgets.HTML(f"""
      <b>Selected Video Folder: </b>{self.content}<br>
      <b>Number of Videos: </b>{self.total_videos}<br>
    """)
    hr = widgets.HTML("<hr>", layout=widgets.Layout(width="300px"))

    vbox_content = []
    if self.stage == 1:
      hbox_btn = widgets.HBox([self.start_button, self.back_button])
      vbox_content = [label_folder, hr, self.TrainingConfig.getForm(), hbox_btn]
    elif self.stage >= 2:
      train_label_1 = widgets.HBox([widgets.HTML(svg_done if self.train_stage[0] else svg_loading),widgets.HTML("<b>Saving config</b>")])
      train_label_2 = widgets.HBox([widgets.HTML(svg_done if self.train_stage[1] else svg_loading),widgets.HTML("<b>Cutting videos</b>")])
      train_label_3 = widgets.HBox([widgets.HTML(svg_done if self.train_stage[2] else svg_loading),widgets.HTML("<b>Generating metadata</b>")])
      train_label_4 = widgets.HBox([widgets.HTML(svg_done if self.train_stage[3] else svg_loading),widgets.HTML("<b>Training model</b>")])

      vbox_content = [label_folder, hr, train_label_1, train_label_2, train_label_3, train_label_4, hr]

      if self.stage == 3:
        label_folder_output = widgets.HTML(f"<b>Custom Model: </b>{self.TrainingConfig.output_dir.value}<br>")
        vbox_content.extend([label_folder_output, self.done_button])

    self.screen.center = widgets.VBox(vbox_content)

  def run_train(self):
    os.makedirs(self.training_branch, exist_ok=True)

    self.stage = 2
    self.build_screen()

    self.TrainingConfig.save_config(self.training_branch)
    self.train_stage[0] = True
    self.build_screen()

    cut_video(self.training_dataset, self.training_branch)
    self.train_stage[1] = True
    self.build_screen()

    generate_training_metadata(self.training_branch)
    self.train_stage[2] = True
    self.build_screen()

    run_model_training()
    self.train_stage[3] = True
    self.build_screen()

    self.stage = 3
    self.build_screen()

  def total_folder_items(self):
    dataset_folder = os.listdir(f"{VIDEO_DIR_PATH}/{self.content}")
    return len([file for file in dataset_folder if any(file.endswith(ext) for ext in ALLOWED_VIDEO_EXT)])



In [41]:
# @title TestState { display-mode: "form" }

class TestState(BaseState):
  def __init__(self, state_manager, content):
    BaseState.__init__(self, state_manager, "Test Model")
    self.content = content
    self.video_folder = Path(f"{VIDEO_DIR_PATH}/{self.content}").as_posix()
    self.videos = os.listdir(self.video_folder)
    self.total_videos = len([file for file in self.videos if any(file.endswith(ext) for ext in ALLOWED_VIDEO_EXT) and not file.startswith('.')])

    self.screen = widgets.AppLayout(
      header=widgets.HTML(value=f"<h1>{self.title}</h1>"),
      left_sidebar=None,
      center=None,
      right_sidebar=None,
      footer=None,
      pane_heights=[1,10,1]
    )
    self.stage = 1
    self.test_stage = [False, False, False, False]

    self.back_button = widgets.Button(description="Back")
    self.back_button.on_click(lambda x: self.state_manager.change_state(State.EXPLORE))

    self.start_button = widgets.Button(description="Start", button_style="info")
    self.start_button.on_click(lambda x: self.run_test())

    self.done_button = widgets.Button(description="Done", button_style="info")
    self.done_button.on_click(lambda x: self.state_manager.change_state(State.EXPLORE))

    self.TestConfig = TestConfig()
    self.build_screen()

  def getScreen(self):
    return self.screen

  def build_screen(self):
    label_folder = widgets.HTML(f"""
      <b>Selected Video Folder: </b>{self.content}<br>
      <b>Number of Videos: </b>{self.total_videos}<br>
    """)
    hr = widgets.HTML("<hr>", layout=widgets.Layout(width="300px"))

    vbox_content = []
    if self.stage == 1:
      hbox_btn = widgets.HBox([self.start_button, self.back_button])
      vbox_content = [label_folder, hr, self.TestConfig.getForm(), hbox_btn]
    elif self.stage >= 2:
      test_label_1 = widgets.HBox([widgets.HTML(svg_done if self.test_stage[0] else svg_loading),widgets.HTML("<b>Saving config</b>")])
      test_label_2 = widgets.HBox([widgets.HTML(svg_done if self.test_stage[1] else svg_loading),widgets.HTML("<b>Running inference</b>")])
      test_label_3 = widgets.HBox([widgets.HTML(svg_done if self.test_stage[2] else svg_loading),widgets.HTML("<b>Retrieving keypoints</b>")])
      test_label_4 = widgets.HBox([widgets.HTML(svg_done if self.test_stage[3] else svg_loading),widgets.HTML("<b>Calculating score</b>")])

      vbox_content = [label_folder, hr, test_label_1, test_label_2, test_label_3, test_label_4, hr]

      if self.stage == 3:
        label_folder_output = widgets.HTML(f"<b>Test: </b>{self.TestConfig.get_output_dir()}<br>")
        vbox_content.extend([label_folder_output, self.done_button])

    self.screen.center = widgets.VBox(vbox_content)

  def test_loop(self):
    video_folder = Path(f"{VIDEO_DIR_PATH}/{self.content}")
    model_name = self.TestConfig.pretrained_model_path.value.as_posix().split("/")[-1]
    skeleton_videos = [file for file in self.videos if any(file.endswith(ext) for ext in ALLOWED_VIDEO_EXT) and not file.startswith('.')]

    for video in skeleton_videos:
      output_dir_folder_name = Path(f"{TEST_OUTPUT_DIR_PATH}/{model_name}/{video[:-4]}")

      self.TestConfig.update_output_dir(output_dir_folder_name.as_posix())

      current_skeleton_path = Path(f"{video_folder}/{video}")
      test_yaml_path = Path(f"{CONFIG_DIR_PATH}/pose_test.yaml").as_posix()

      # Start inference
      !accelerate launch txt2video.py \
          --config={test_yaml_path}  \
          --skeleton_path={current_skeleton_path}

      # Retrieve info needed from the config file
      video_length = self.TestConfig.video_length.value

      # Create a pose gif of all 3 types of pose (Human, Skeleton, Superimposed)
      postprocess_mmpose(current_skeleton_path, video_length, output_dir_folder_name)

      # Superimposes the pose onto generated gifs
      superimposeSkeleton(output_dir_folder_name)

      # Converts the raw gifs back to mp4 to get keypoints
      convertGifs(output_dir_folder_name)

      # Post process gifs
      postprocess_gif_test(model_name)

  def post_test_loop(self):
    model_name = self.TestConfig.pretrained_model_path.value.as_posix().split("/")[-1]
    model_path = Path(f"{TEST_OUTPUT_DIR_PATH}/{model_name}")
    test_folders = [f for f in os.listdir(model_path) if os.path.isdir(os.path.join(model_path, f))]

    %cd -q {MMPOSE_DIR_PATH}

    for folder in test_folders:
      if folder.startswith("."):
        continue

      mp4_path = Path(f"{model_path}/{folder}/gif_to_video")
      videos_to_process = [f for f in os.listdir(mp4_path) if os.path.isfile(os.path.join(mp4_path, f))]

      test_pose_path = Path(f"{model_path}/{folder}")

      generated_path = Path(f"{test_pose_path}/generated_poses")
      keypoints_path = Path(f"{test_pose_path}/keypoints")

      os.makedirs(generated_path, exist_ok=True)
      os.makedirs(keypoints_path, exist_ok=True)

      for video in videos_to_process:
        processed_video_path = Path(f"{mp4_path}/'{video}'")

        # Start inference with black backgroud
        !python demo/inferencer_demo.py \
            {processed_video_path}  \
            --pose2d human \
            --black-background \
            --thickness 4 \
            --radius 0 \
            --vis-out-dir {generated_path} \
            --pred-out-dir {keypoints_path}

  def test_results(self):
    model_name = self.TestConfig.pretrained_model_path.value.as_posix().split("/")[-1]
    model_path = Path(f"{TEST_OUTPUT_DIR_PATH}/{model_name}")
    test_folders = [f for f in os.listdir(model_path) if os.path.isdir(os.path.join(model_path, f))]
    csv_path = Path(f"{model_path}/score.csv")

    %cd -q {MMPOSE_DIR_PATH}

    for folder in test_folders:
      keypoints_path = Path(f"{model_path}/{folder}/keypoints")
      pose_human_path = Path(f"{keypoints_path}/pose-human.json")

      with open(pose_human_path, 'r') as json_file:
        pose_human_json = json.load(json_file)

      for kp_file in os.listdir(keypoints_path):
        if kp_file == "pose-human.json":
          continue

        pose_inf_path = Path(f"{keypoints_path}/{kp_file}")
        with open(pose_inf_path, 'r') as json_file:
          pose_inf_json = json.load(json_file)

        score = calculate_score(pose_human_json, pose_inf_json)
        create_or_append_csv(csv_path, [folder, kp_file[:-5], score, ""])


  def run_test(self):
    self.stage = 2
    self.build_screen()

    self.TestConfig.save_config()
    self.test_stage[0] = True
    self.build_screen()

    self.test_loop()
    self.test_stage[1] = True
    self.build_screen()

    self.post_test_loop()
    self.test_stage[2] = True
    self.build_screen()

    self.test_results()
    self.test_stage[3] = True
    self.build_screen()

    self.stage = 3
    self.build_screen()




## 🖥️ **App** 🖥️

In [45]:
# @title <font size="-10">🖥️ Beta v2.0</font> { display-mode: "form" }
state_manager = StateManager()
state_manager.start()

HTML(value='<style>.check {background-color: white;border-radius: 100%;fill: green;margin: 2.5px 6px 2.5px 0px…

Output()